In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scikit-learn pandas numpy matplotlib seaborn imblearn scipy

### Attach google drive


## Import all the necessary libraries and packages


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import scipy
import random

In [10]:
data_train = pd.read_csv('/content/drive/MyDrive/sleep onset datasets/data_train_cleaned_sbjs.csv')
data_test = pd.read_csv('/content/drive/MyDrive/sleep onset datasets/data_test_cleaned_sbjs.csv')


In [11]:
data_all = pd.concat([data_train, data_test], axis=0)
print(data_all.shape)

(165000, 90)


In [12]:
data_test

,Label,Sbj_ID,Age,Gender,Race1,Race2,Race3,Race4,ifCleanOnset,Time2Sleep,...,wrseltr5,slpapnea5,cpap5,dntaldv5,uvula5,insmnia5,rstlesslgs5,whiirs5c,epslpscl5c,hoostmeq5c
0,29.9,77,61,1,0.0,0.0,0.0,1.0,1,37.5,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,10.0
1,29.8,77,61,1,0.0,0.0,0.0,1.0,1,37.5,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,10.0
2,29.7,77,61,1,0.0,0.0,0.0,1.0,1,37.5,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,10.0
3,29.6,77,61,1,0.0,0.0,0.0,1.0,1,37.5,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,10.0
4,29.5,77,61,1,0.0,0.0,0.0,1.0,1,37.5,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16495,0.4,6739,77,0,1.0,0.0,0.0,0.0,0,62.5,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0,10.0
16496,0.3,6739,77,0,1.0,0.0,0.0,0.0,0,62.5,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0,10.0
16497,0.2,6739,77,0,1.0,0.0,0.0,0.0,0,62.5,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0,10.0
16498,0.1,6739,77,0,1.0,0.0,0.0,0.0,0,62.5,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0,10.0


In [13]:
# plot how many nans there are for each participant in data_test

test_unique_ids = data_test['Sbj_ID'].unique()
test_unique_ids = test_unique_ids.tolist()
len(test_unique_ids)

# calculate how many nans there are for each participant in data_test
nans_per_participant = {}
for i in test_unique_ids:
    nans_per_participant[i] = data_test[data_test['Sbj_ID'] == i]['delta'].isnull().sum()

# get the ids of the pariticpants with the least nans
nans_per_participant = {k: v for k, v in sorted(nans_per_participant.items(), key=lambda item: item[1])}
print(nans_per_participant)

# get the top 20 participants with the least nans
top_20 = list(nans_per_participant.keys())[:20]

# get data_test with only the top 20 participants with the least nans
selected_participants_data = data_test[data_test['Sbj_ID'].isin(top_20)]

# save the data_test with only the top 20 participants with the least nans
selected_participants_data.to_csv('data_test_cleaned_sbjs_top_20.csv', index=False)


{2251: 1, 4512: 3, 188: 5, 5801: 7, 3689: 9, 4618: 9, 4998: 9, 6279: 10, 2780: 12, 933: 14, 5255: 14, 2443: 15, 1077: 18, 5568: 18, 2574: 21, 912: 22, 6321: 22, 6475: 22, 110: 25, 2019: 25, 6416: 25, 977: 26, 3801: 27, 2616: 28, 6414: 28, 2167: 29, 6261: 29, 1021: 30, 1385: 30, 2452: 30, 5887: 30, 1951: 31, 5882: 31, 4186: 35, 4806: 35, 3347: 36, 6240: 37, 916: 38, 6739: 40, 949: 41, 2750: 42, 4582: 42, 962: 44, 4969: 45, 3708: 47, 1717: 48, 77: 51, 4038: 53, 5354: 67, 1033: 69, 2897: 75, 4393: 75, 2881: 85, 4756: 88, 2191: 104}


In [14]:
selected_participants_data = pd.read_csv('/content/drive/MyDrive/sleep onset datasets/data_test_cleaned_sbjs_top_20.csv')
selected_participants = selected_participants_data['Sbj_ID'].unique().tolist()
selected_participants

[110,
 188,
 912,
 933,
 1077,
 2019,
 2251,
 2443,
 2574,
 2780,
 3689,
 4512,
 4618,
 4998,
 5255,
 5568,
 5801,
 6279,
 6321,
 6475]

## Define the helper functions

### Define fuctions to create custom Dataset and DataLoader


In [15]:
# Define the device:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Create a PyTorch dataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class SleepDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return torch.from_numpy(self.sequences[idx]).float(), torch.from_numpy(np.array(self.labels[idx])).float()

### Define functions to preprocess the data -> break up into sequences, resample, stratify, etc.

In [16]:
# function to turn dataset from regression to classification
def regression_to_classification(dataframe, sleep_onset_threshold, awake_threshold):
    ''' This function takes a dataframe with regression labels and turns it into a classification problem '''

    # Try classification on 5 minutes before sleep and 20+ minutes before sleep
    dataframe_transformed = dataframe[(dataframe['Label'] <= sleep_onset_threshold) | (dataframe['Label'] > awake_threshold)].copy()

    # Swap Label columns into 1  (if Label <= 5) and 0
    dataframe_transformed['Label'] = dataframe_transformed['Label'].apply(lambda x: 1 if x <= sleep_onset_threshold else 0)

    index = [1, 0]
    label = [f'{sleep_onset_threshold} minutes', f'{awake_threshold}-30 minutes']
    return dataframe_transformed, index, label


In [17]:
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

def handle_missing_data(data, ifmissing):
    if not ifmissing:
        assert data.isnull().sum().sum() == 0, "There are NaNs in the data"

def ss_split_remaining_subjects(subjects, labels, random_seed):
        sss_val_test = StratifiedShuffleSplit(n_splits=1, train_size=0.5,
                                              test_size=0.5, random_state=random_seed)
        for val_idx, test_idx in sss_val_test.split(subjects, labels):
            val_subjects = subjects[val_idx]
            test_subjects = subjects[test_idx]
        return val_subjects, test_subjects


def split_data_into_sets(data, train_proportion, if_stratified_sampling, iftest, random_seed):
     # Create a list of all unique subjects
    subjects = data['Sbj_ID'].unique()

    # Randomly select some of the 80% of subjects to be in the training set, 20% to the validation set and 20% in the test set
    np.random.seed(random_seed)

    val_subjects = []
    test_subjects = []
    train_subjects = []
    if if_stratified_sampling == 1:
        # Get representative labels for each subject for stratification
        subject_labels = data.groupby('Sbj_ID')['Label'].apply(lambda x: x.value_counts().idxmax())

        sss = StratifiedShuffleSplit(n_splits=1, train_size=train_proportion, test_size=1-train_proportion, random_state=random_seed)

        for train_idx, temp_idx in sss.split(subjects, subject_labels):
            train_subjects = subjects[train_idx]
            temp_subjects = subjects[temp_idx]

        if iftest == 1:
            # Now split the remaining subjects into validation and test sets
            sss_val_test = StratifiedShuffleSplit(n_splits=1, train_size=0.5, test_size=0.5, random_state=random_seed)

            for val_idx, test_idx in sss_val_test.split(temp_subjects, subject_labels.iloc[temp_idx]):
                val_subjects = temp_subjects[val_idx]
                test_subjects = temp_subjects[test_idx]
        else:
            val_subjects = temp_subjects
    else:
        train_subjects = np.random.choice(subjects, size=int(train_proportion*len(subjects)), replace=False)


        if iftest == 1:
            val_proportions = (1 - train_proportion)/2
            val_subjects = np.random.choice(np.setdiff1d(subjects, train_subjects), size=int(val_proportions*len(subjects)), replace=False)
            test_subjects = np.setdiff1d(subjects, np.concatenate((train_subjects, val_subjects)))
        else:
            val_proportions = 1 - train_proportion
            val_subjects = np.setdiff1d(subjects, train_subjects)
            test_subjects = []

    # Print the number of subjects in each set
    print(f'There are {len(train_subjects)} subjects in the training set, {len(val_subjects)} subjects in the validation set and {len(test_subjects)} subjects in the test set')

    return train_subjects, val_subjects, test_subjects

def assign_data_set(data, train_subjects, val_subjects, test_subjects):
    data['Set'] = 'train'
    data.loc[data['Sbj_ID'].isin(val_subjects), 'Set'] = 'val'
    data.loc[data['Sbj_ID'].isin(test_subjects), 'Set'] = 'test'
    return data.sort_values('Sbj_ID')

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

def generate_sequences_and_labels(group, window_size, ifoutput_end_points=0):
    sequences = []
    labels = []

    group_features = group.drop(['Sbj_ID', 'Label', 'Set'], axis=1).to_numpy()
    ifmissing_column = group['ifmissing_after_imputation'].to_numpy()

    # If window size is longer than the group, skip
    if window_size > len(group_features):
        return sequences, labels

    for i in range(len(group_features) - window_size):
        if ifmissing_column[i : i + window_size].sum() > 0:
            continue

        if ifoutput_end_points == 0 and group['Label'].iloc[i : i + window_size].nunique() > 1:
            continue

        sequences.append(group_features[i : i + window_size])
        labels.append(group['Label'].iloc[i])

    return sequences, labels

def resample_data(sequences, labels, method, window_size, random_seed=42):
    if method == 'oversampling':
        print('Performing SMOTE oversampling')
        smote = SMOTE(random_state=random_seed)
        sequences_resampled, labels_resampled = smote.fit_resample(
            sequences.reshape(sequences.shape[0], -1), labels)
        sequences = sequences_resampled.reshape(-1, window_size, sequences.shape[-1])
        labels = labels_resampled
    elif method == 'undersampling':
        print('Performing undersampling')
        rus = RandomUnderSampler(random_state=random_seed)
        sequences_resampled, labels_resampled = rus.fit_resample(
            sequences.reshape(sequences.shape[0], -1), labels)
        sequences = sequences_resampled.reshape(-1, window_size, sequences.shape[-1])
        labels = labels_resampled

    return sequences, labels

def determine_output_end_points(train_proportion, iftest, train_end_points, test_end_points, val_end_points):
    if int(train_proportion) == 1:
        return train_end_points
    else:
        if iftest == 1:
            return test_end_points
        else:
            return val_end_points

def format_output(train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels, ifoutputsubjects, train_subjects, val_subjects, test_subjects, ifoutput_end_points, end_points=None, sleep_stages=None):

    if ifoutputsubjects:

        if ifoutput_end_points == 1:
            return train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels, train_subjects, val_subjects, test_subjects, end_points, sleep_stages
        else:
            return train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels, train_subjects, val_subjects, test_subjects

    else:
        if ifoutput_end_points == 1:
            return train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels, end_points, sleep_stages
        else:
            return train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels


In [18]:
def create_sliding_windows_new(data, window_size_minutes=0.5, ifmissing=True, random_seed=42, ifoutputsubjects=0,
                           train_proportion=0.8, ifoutput_end_points=0, if_stratified_sampling=1,
                           resampling_method=None, iftest=1):

    handle_missing_data(data, ifmissing)

    train_subjects, val_subjects, test_subjects = split_data_into_sets(data, train_proportion, if_stratified_sampling, iftest, random_seed)

    data = assign_data_set(data, train_subjects, val_subjects, test_subjects)

    # Convert the window size from minutes to 6-second epochs
    window_size = int((window_size_minutes * 60) / 6)

    train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels, train_end_points, train_sleep_stages, val_end_points, val_sleep_stages, test_end_points, test_sleep_stages = generate_sequences_and_labels(data, window_size, ifoutput_end_points)

    train_sequences, train_labels = resample_data(train_sequences, train_labels, resampling_method, random_seed, window_size)

    if ifoutput_end_points:
        end_points, sleep_stages = determine_output_end_points(train_proportion, iftest, train_end_points, val_end_points, test_end_points, train_sleep_stages, val_sleep_stages, test_sleep_stages)

    return format_output(train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels, ifoutputsubjects, ifoutput_end_points, train_subjects, val_subjects, test_subjects, end_points, sleep_stages)

In [19]:


# Create a function for creating sliding windows of selected length
def create_sliding_windows(data,  window_size_minutes=0.5,
                           ifmissing = True, random_seed = 42, ifoutputsubjects = 0,
                           train_proportion = 0.8, ifoutput_end_points = 0,
                           if_stratified_sampling = 1,
                           resampling_method = None, iftest = 0, ifvocal = False):

    # Assert whether therea are any NaNs in the data
    if not ifmissing:
        assert data.isnull().sum().sum() == 0, "There are NaNs in the data"

    # extract 'Old_label' column
    labels = data['Old_label'].to_numpy()

    # Create a list of all unique subjects
    subjects = data['Sbj_ID'].unique()

    # Randomly select some of the 80% of subjects to be in the training set, 20% to the validation set and 20% in the test set
    np.random.seed(random_seed)

    val_subjects = []
    test_subjects = []
    train_subjects = []
    if if_stratified_sampling == 1:
        # Get representative labels for each subject for stratification
        subject_labels = data.groupby('Sbj_ID')['Label'].apply(lambda x: x.value_counts().idxmax())

        sss = StratifiedShuffleSplit(n_splits=1, train_size=train_proportion, test_size=1-train_proportion, random_state=random_seed)

        for train_idx, temp_idx in sss.split(subjects, subject_labels):
            train_subjects = subjects[train_idx]
            temp_subjects = subjects[temp_idx]

        if iftest == 1:
            # Now split the remaining subjects into validation and test sets
            sss_val_test = StratifiedShuffleSplit(n_splits=1, train_size=0.5, test_size=0.5, random_state=random_seed)

            for val_idx, test_idx in sss_val_test.split(temp_subjects, subject_labels.iloc[temp_idx]):
                val_subjects = temp_subjects[val_idx]
                test_subjects = temp_subjects[test_idx]
        else:
            val_subjects = temp_subjects
    else:
        train_subjects = np.random.choice(subjects, size=int(train_proportion*len(subjects)), replace=False)


        if iftest == 1:
            val_proportions = (1 - train_proportion)/2
            val_subjects = np.random.choice(np.setdiff1d(subjects, train_subjects), size=int(val_proportions*len(subjects)), replace=False)
            test_subjects = np.setdiff1d(subjects, np.concatenate((train_subjects, val_subjects)))
        else:
            val_proportions = 1 - train_proportion
            val_subjects = np.setdiff1d(subjects, train_subjects)
            test_subjects = []

    # Print the number of subjects in each set
    if ifvocal:
        print(f'There are {len(train_subjects)} subjects in the training set, {len(val_subjects)} subjects in the validation set and {len(test_subjects)} subjects in the test set')


    # Create a new column in the dataframe that indicates whether the subject is in the training set, val set or the test set
    data['Set'] = 'train'
    data.loc[data['Sbj_ID'].isin(val_subjects), 'Set'] = 'val'
    data.loc[data['Sbj_ID'].isin(test_subjects), 'Set'] = 'test'

    # Sort your dataframe by Sbj_ID if not already sorted
    data = data.sort_values('Sbj_ID')

    # Convert the window size from minutes to 6-second epochs
    window_size = int((window_size_minutes*60)/6)

    # Create empty lists to store your sequences and labels
    train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels = [], [], [], [], [], []
    if ifoutput_end_points == 1:
        test_end_points, test_sleep_stages, val_end_points, val_sleep_stages, train_end_points, train_sleep_stages = [], [], [], [], [], []

    # Group the DataFrame by subject ID and iterate over each group, dividing into training and test sets
    for _, group in data.groupby('Sbj_ID'):

        group = group.sort_values('Old_label', ascending=False)

        # For each group, get the feature columns and convert them into numpy array
        if ifoutput_end_points == 1:
             group_features = group.drop(['Sbj_ID', 'Label', 'Set', 'ifmissing_after_imputation', 'Old_label', 'ifCleanOnset', 'Time2Sleep', 'SleepStage'], axis=1).to_numpy()
             group_old_labels = group['Old_label'].to_numpy()
             group_sleep_stages = group['SleepStage'].to_numpy()
             group_labels = group['Label'].to_numpy()
        else:
            group_features = group.drop(['Sbj_ID', 'Label', 'Set', 'ifmissing_after_imputation', 'ifCleanOnset', 'Time2Sleep', 'SleepStage'], axis=1).to_numpy()

        ifmissing_column = group['ifmissing_after_imputation'].to_numpy()

        train_test = group['Set'].iloc[0]

        #if window_size == len(group_features):
        #    if train_test == 'train':
        #        print('got here')
        #        train_sequences.append(group_features)
        #        train_labels.append(group['Label'].iloc[0])
        #    elif train_test == 'val':
        #        val_sequences.append(group_features)
        #        val_labels.append(group['Label'].iloc[0])
        #    elif train_test == 'test':
        #        test_sequences.append(group_features)
        #        test_labels.append(group['Label'].iloc[0])
        #elif window_size > len(group_features):
        #    continue


        # Iterate over the group array with a sliding window
        for i in range(len(group_features) - window_size):

            # If there are any NaNs in the window, skip it

            if ifmissing_column[i : i + window_size].sum() > 0:
                continue

            # Check if the labels are consistent in the window (only if we're not outputting probabilities for the whole duration)
            if ifoutput_end_points == 0:
                if group['Label'].iloc[i : i + window_size].nunique() > 1:
                    continue

            if train_test == 'train':
                # Append the window data to your sequences
                train_sequences.append(group_features[i : i + window_size])
                # Append the label corresponding to the end of the window
                train_labels.append(group['Label'].iloc[i])

                if ifoutput_end_points == 1:
                    # Append the starting point of the window to the list of starting points
                    train_end_points.append(group_old_labels[i+window_size])
                    train_sleep_stages.append(group_sleep_stages[i+window_size])

            elif train_test == 'val':
                # Append the window data to your sequences
                val_sequences.append(group_features[i : i + window_size])

                # Append the label corresponding to the end of the window
                val_labels.append(group['Label'].iloc[i])

                if ifoutput_end_points == 1:
                    # Append the starting point of the window to the list of starting points
                    val_end_points.append(group_old_labels[i+window_size])
                    val_sleep_stages.append(group_sleep_stages[i+window_size])


            elif train_test == 'test':

                # Append the window data to your sequences
                test_sequences.append(group_features[i : i + window_size])

                # Append the label corresponding to the end of the window
                test_labels.append(group['Label'].iloc[i])

                if ifoutput_end_points == 1:
                    # Append the starting point of the window to the list of starting points
                    test_end_points.append(group_old_labels[i+window_size])
                    test_sleep_stages.append(group_sleep_stages[i+window_size])

    # Convert the sequences and labels into numpy arrays
    train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels = np.array(train_sequences), np.array(train_labels), np.array(val_sequences), np.array(val_labels), np.array(test_sequences), np.array(test_labels)
    if ifoutput_end_points == 1:
        train_end_points, train_sleep_stages, val_end_points, val_sleep_stages, test_end_points, test_sleep_stages = np.array(train_end_points), np.array(train_sleep_stages), np.array(val_end_points), np.array(val_sleep_stages), np.array(test_end_points), np.array(test_sleep_stages)


    if resampling_method == 'oversampling':
        if ifvocal:
            print('Performing SMOTE oversampling')
        smote = SMOTE(random_state=random_seed)
        if ifoutput_end_points == 1:
            train_sequences_and_end_points = np.concatenate((train_sequences.reshape(train_sequences.shape[0], -1), train_end_points.reshape(-1, 1)), axis=1)
            train_sequences_and_end_points_resampled, train_labels_resampled = smote.fit_resample(train_sequences_and_end_points, train_labels)
            train_sequences_resampled = train_sequences_and_end_points_resampled[:, :-1]
            train_end_points_resampled = train_sequences_and_end_points_resampled[:, -1]
            train_sequences = train_sequences_resampled.reshape(-1, window_size, train_sequences.shape[-1])
            train_end_points = train_end_points_resampled
        else:
            train_sequences_resampled, train_labels_resampled = smote.fit_resample(train_sequences.reshape(train_sequences.shape[0], -1), train_labels)
        train_sequences = train_sequences_resampled.reshape(-1, window_size, train_sequences.shape[-1])
        train_labels = train_labels_resampled
    elif resampling_method == 'undersampling':
        if ifvocal:
            print('Performing undersampling')
        rus = RandomUnderSampler(random_state=random_seed)
        if ifoutput_end_points == 1:
            train_sequences_and_end_points = np.concatenate((train_sequences.reshape(train_sequences.shape[0], -1), train_end_points.reshape(-1, 1)), axis=1)
            train_sequences_and_end_points_resampled, train_labels_resampled = rus.fit_resample(train_sequences_and_end_points, train_labels)
            train_sequences_resampled = train_sequences_and_end_points_resampled[:, :-1]
            train_end_points_resampled = train_sequences_and_end_points_resampled[:, -1]
            train_sequences = train_sequences_resampled.reshape(-1, window_size, train_sequences.shape[-1])
            train_end_points = train_end_points_resampled
        else:
            train_sequences_resampled, train_labels_resampled = rus.fit_resample(train_sequences.reshape(train_sequences.shape[0], -1), train_labels)
        train_sequences = train_sequences_resampled.reshape(-1, window_size, train_sequences.shape[-1])
        train_labels = train_labels_resampled
    elif resampling_method is None:
        pass

    if ifoutput_end_points == 1:
        if int(train_proportion) == 1:

            end_points = train_end_points
            sleep_stages = train_sleep_stages
            if ifvocal:
                print('end_points shape', end_points.shape)

        else:
            if iftest == 1:
                end_points = test_end_points
                sleep_stages = test_sleep_stages
            else:
                end_points = val_end_points
                sleep_stages = val_sleep_stages

    del data
    gc.collect()


    if ifoutputsubjects:
        if ifoutput_end_points == 1:
            return train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels, train_subjects, val_subjects, test_subjects, end_points, sleep_stages
        else:
            return train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels, train_subjects, val_subjects, test_subjects
    else:
        if ifoutput_end_points == 1:
            return train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels, end_points, sleep_stages
        else:
            return train_sequences, train_labels, val_sequences, val_labels, test_sequences, test_labels




In [20]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Create a function to impute the missing values in the dataset
def impute_missing_values (dataframe, method = 'LOCF', limit = 20):

    # create a missing mask (column) fthat would indicate whether the values in any of the columns of mydata are missing
    # 1 indicates missing, 0 indicates not missing
    missing_mask = dataframe.isnull().sum(axis=1).astype(bool).astype(int)
    dataframe_imputed = dataframe.copy()
    del dataframe
    dataframe_imputed = pd.concat([dataframe_imputed, missing_mask.rename('ifmissing')], axis=1)

    if method == 'LOCF':

        dataframe_imputed.fillna(method='ffill', inplace=True, limit = limit)

        # fill the rest with backward fill
        #dataframe_imputed.fillna(method='bfill', inplace=True, limit = limit)

    elif method =='NOCB':

        dataframe_imputed.fillna(method='bfill', inplace=True, limit = limit)

        # fill the rest with forward fill
        #dataframe_imputed.fillna(method='ffill', inplace=True, limit = limit)

    elif method == 'linear interpolation':

        dataframe_imputed.interpolate(method='linear', inplace=True, limit = limit)

    elif method == 'quadratic interpolation':

        dataframe_imputed.interpolate(method='quadratic', inplace=True, limit = limit)

    elif method == 'mean':

        dataframe_imputed.fillna(dataframe.mean(), inplace=True, limit = limit)

    elif method == 'median':

        dataframe_imputed.fillna(dataframe.median(), inplace=True, limit = limit)

    elif method == 'MICE':

        # Define an imputer
        imp = IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, random_state=0),
                            missing_values=np.nan,
                            sample_posterior=False,
                            max_iter=10,
                            random_state=0,
                            verbose=2)


        columns = dataframe_imputed.columns

        # Apply the imputer
        df_imputed = imp.fit_transform(dataframe_imputed)

        # Convert back to DataFrame
        dataframe_imputed = pd.DataFrame(df_imputed, columns=columns)

    elif method == 'None':
        dataframe_imputed = dataframe_imputed

    missing_mask_new = dataframe_imputed.isnull().sum(axis=1).astype(bool).astype(int)

    dataframe_imputed = pd.concat([dataframe_imputed, missing_mask_new.rename('ifmissing_after_imputation')], axis=1)

    return dataframe_imputed


### Define the model architectures

In [21]:
import torch
import torch.nn as nn

class SleepOnsetRNNClassifier(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, output_size=1, dropout=0.0, l2=0.0):
        super(SleepOnsetRNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.l2 = l2

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
       #prob = self.sigmoid(out)
        return out


In [22]:
class PretrainedModelForRegression(nn.Module):
    def __init__(self, original_model):
        super(PretrainedModelForRegression, self).__init__()

        # Extract feature parts from the original model
        self.features = nn.Sequential(
            original_model.lstm,
            # You can add more layers from the original model here if needed
        )

        # Regression head
        self.regression_head = nn.Linear(original_model.hidden_size, 1)

    def forward(self, x):
        # Note: Here we're not initializing h0 and c0 as the original model does.
        # This means they will default to 0s.
        out, _ = self.features(x)
        out = self.regression_head(out[:, -1, :])
        return out

### Define the training and evaluation functions for classification

In [23]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train function
def train_classification(model, train_loader, val_loader, num_epochs,
                        criterion, optimizer, device, index,
                        threshold=0.5, ifprobabilities=False, ifplot = False):

    train_losses = []
    val_losses = []
    sigmoid_function = torch.nn.Sigmoid()  # define a sigmoid function

    for epoch in range(num_epochs):

        # Train the model
        model.train()
        train_predictions, train_actuals = [], []
        for i, (inputs, labels_and_endpoints) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels_and_endpoints[:, 0].to(device)


            # Forward pass
            outputs = model(inputs)
            #print(outputs)
            #print(labels)
            outputs_detach = outputs.detach()
            #print(outputs_detach)
            probabilities = sigmoid_function(outputs_detach).cpu().numpy() # convert logits to probabilities
            #print(probabilities)

            loss = criterion(outputs, labels.view(-1, 1))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_actuals.extend(labels.cpu().numpy())
            train_predictions.extend((probabilities > threshold).astype(int).flatten())


        report = classification_report(train_actuals, train_predictions)
        if ifplot:
            print('-'*100)
            print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item()}')
            print('-'*100)
            print('Training Report:')
            print(report)
            print('-'*100)

        # Validate the model
        model.eval()
        with torch.no_grad():
            val_loss = 0
            predictions, actuals, prediction_probabilities = [], [], []
            for inputs, labels_and_endpoints in val_loader:
                inputs = inputs.to(device)
                labels = labels_and_endpoints[:, 0].to(device)

                logits = model(inputs)  # these are logits now
                val_loss += criterion(logits, labels.view(-1, 1)).item()

                probabilities = sigmoid_function(logits).cpu().numpy()  # convert logits to probabilities

                predictions.extend((probabilities > threshold).astype(int).flatten())


                if ifprobabilities:
                    prediction_probabilities.extend(probabilities)

                actuals.extend(labels.cpu().numpy())


            # Calculate metrics
            f1_scores = f1_score(actuals, predictions, average='weighted', labels = index)
            if ifplot:
                print('-'*100)
                print(f'Validation Loss: {val_loss/len(val_loader)}')
                print('F1-Scores for each class:')
                print(f1_scores)

                # Calculate metrics
                report = classification_report(actuals, predictions)

                print('Validation Report:')
                print(report)


        # Save the model training and validation losses for plotting
        train_losses.append(loss.item())
        val_losses.append(val_loss/len(val_loader))


    del train_predictions, train_actuals

    if ifprobabilities:
        return train_losses, val_losses, model, optimizer, predictions, actuals, prediction_probabilities
    else:
        return train_losses, val_losses, model, optimizer, predictions, actuals


# Test function
def test_classification(model, test_loader, device, criterion, index, threshold=0.5, ifprobabilities=False, ifplot = False):
    model.eval()
    predictions, actuals, prediction_probabilities = [], [], []

    sigmoid_function = torch.nn.Sigmoid()  # define a sigmoid function

    with torch.no_grad():
        test_loss = 0
        for inputs, labels_and_endpoints in test_loader:
            inputs = inputs.to(device)
            labels = labels_and_endpoints[:, 0].to(device)

            logits = model(inputs)  # these are logits now
            test_loss += criterion(logits, labels.view(-1, 1)).item()

            probabilities = sigmoid_function(logits).cpu().numpy()  # convert logits to probabilities

            predictions.extend((probabilities > threshold).astype(int).flatten())
            if ifprobabilities:
                prediction_probabilities.extend(probabilities)
            actuals.extend(labels.cpu().numpy())
        if ifplot:
            print(f'Test Loss: {test_loss/len(test_loader)}')

    # Calculate metrics
    f1_scores = f1_score(actuals, predictions, average=None, labels=index)


    if ifplot:
        print('F1-Scores for each class:')
        print(f1_scores)

    # Calculate metrics
    report = classification_report(actuals, predictions)

    if ifprobabilities:
        return predictions, actuals, prediction_probabilities
    else:
        return predictions, actuals

In [24]:

def get_timeline(mydata_no_nan):

    # assert that the data has no NaNs
    assert mydata_no_nan.isnull().sum().sum() == 0, 'There are NaNs in the data'


    # Create X and y from dataframes to use in scikit-learn (drop Label, SleepStage, Sbj_ID and ifCleanOnset columns)
    X_all = mydata_no_nan.drop(['Label', 'Old_label', 'SleepStage', 'Sbj_ID', 'ifCleanOnset', 'Time2Sleep'], axis=1)
    y_all = mydata_no_nan['Label']
    y_old_all = mydata_no_nan['Old_label']
    y_sleepstage_all = mydata_no_nan['SleepStage']

    del mydata_no_nan



    return X_all, y_all, y_old_all, y_sleepstage_all

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import os
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                             roc_curve, roc_auc_score, auc)
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score

def evaluate_classification(true_labels, predictions, prediction_probabilities,
                            index=['10 minutes before sleep', 'Awake'],
                            columns=['10 minutes before sleep', 'Awake'],
                            save_path='confusion_matrix.png', ifsaveplots = False,
                            output_path=None, iftest = False, ifplot = True, classes = None, ifvocal = True):

    # Evaluate the performance of the model
    accuracy = accuracy_score(true_labels, predictions)
    if iftest:
        if ifvocal:
            print(f'Accuracy: {accuracy}')

    # Classification report
    report = classification_report(true_labels, predictions)
    if iftest:
        if ifvocal:
            print(report)

    # Decide on the filename based on iftest
    report_filename = 'test_classification_report.txt' if iftest else 'crossval_classification_report.txt'

     # Dictionary to store all metrics
    metrics_dict = {}
    metrics_dict['accuracy'] = accuracy
    # Classification report
    precision, recall, _,  _ = precision_recall_fscore_support(true_labels, predictions, average='binary')
    f1 = f1_score(true_labels, predictions, average='weighted')
    f1_macro = f1_score(true_labels, predictions, average='macro')
    f1_pos, f1_neg = f1_score(true_labels, predictions, average=None)


    metrics_dict['precision'] = precision
    metrics_dict['recall'] = recall
    metrics_dict['f1_weighted'] = f1
    metrics_dict['f1_macro'] = f1_macro
    metrics_dict['f1_pre_sleep'] = f1_pos
    metrics_dict['f1_awake'] = f1_neg



    if iftest:
        if ifvocal:
            print('F1 pre-sleep: ', f1_pos)
            print('F1 awake: ', f1_neg)



    # Save classification report to a .txt file
    if ifsaveplots:
        if output_path:
            report_filename = os.path.join(output_path, report_filename)
        with open(report_filename, 'w') as f:
            f.write(report)
    # Plot the confusion matrix

    # Include the names of the classes in the confusion matrix
    cm = confusion_matrix(true_labels, predictions, normalize='true', labels=index)
    cm = pd.DataFrame(cm, index=index, columns=columns)

    # Rename the columns and rows to the names of the classes
    cm.index = columns
    plt.figure(figsize=(5.5, 4))
    sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues')

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix for times before sleep')

    if iftest:
        saving_prefix = 'test'
    else:
        saving_prefix = 'crossval'

    save_path = f'{saving_prefix}_{save_path}'

    if ifsaveplots:
        if output_path:
            save_path = os.path.join(output_path, save_path)
        # Save the confusion matrix to a file
        plt.savefig(save_path)

    # Display the confusion matrix
    if ifplot:
        plt.show()
    plt.close()

    roc_path = f'{saving_prefix}_roc_curve.png'

    # ROC and AUC

    # if array is 2D
    if type(prediction_probabilities) == list:
        prediction_probabilities = np.array(prediction_probabilities)
        prediction_probabilities = prediction_probabilities.ravel()


    fpr, tpr, _ = roc_curve(true_labels, prediction_probabilities)
    roc_auc = auc(fpr, tpr)
    #metrics_dict['roc_curve'] = (fpr, tpr)
    metrics_dict['auc'] = roc_auc

    plt.figure(figsize=(7, 7))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")


    if ifsaveplots:
        if output_path:
            roc_path = os.path.join(output_path, roc_path)
        plt.savefig(roc_path)

    if ifplot:
        plt.show()
    plt.close()

    del true_labels, predictions, cm, report, f1, f1_macro, f1_pos, f1_neg, roc_auc



    return metrics_dict


### Define the functions for probability analysis

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import os

def preprocess_data(y_pred, y_true, timeline, sleep_stage):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    timeline = np.array(timeline)
    sleep_stage = np.array(sleep_stage)
    timeline = np.round(timeline, 1)
    return y_pred, y_true, timeline, sleep_stage

def get_indices_for_unique_values(timeline):
    unique_values = np.unique(timeline)
    unique_values_sorted = np.sort(unique_values)
    indices = {}
    for value in unique_values:
        indices[value] = np.where(timeline == value)[0]
    return dict(sorted(indices.items())), unique_values_sorted

def update_indices_with_missing_values(indices, rnn_window):
    timegrid = np.arange(0, 30.1 - rnn_window, 0.1)
    timegrid = np.round(timegrid, 1)
    missing_values = np.setdiff1d(timegrid, list(indices.keys()))
    for value in missing_values:
        indices[value] = np.nan
    return dict(sorted(indices.items())), timegrid

def sort_predictions_by_time(y_pred, indices, timegrid):
    y_pred_sorted = [np.mean(y_pred[indices[value]]) if not np.isnan(indices[value]).any() else np.nan for value in timegrid]
    y_pred_std = [np.std(y_pred[indices[value]]) if not np.isnan(indices[value]).any() else np.nan for value in timegrid]
    return y_pred_sorted, y_pred_std

def downsample_data(data, window_for_averaging):
    return [np.nanmean(data[i:i+window_for_averaging]) for i in range(0, len(data), window_for_averaging)]

def plot_data(y_pred_mean, y_pred_std, sleep_stage_mean, sleep_stage_std, timegrid, window_for_averaging, rnn_window, pre_sleep_window=None, awake_window=None, ifsaveplots=False, output_path=None, ifplot = False, ifsubjecttest = False, change_points_times = None):

    # convert to numpy arrays to avoid errors
    y_pred_mean = np.array(y_pred_mean)
    y_pred_std = np.array(y_pred_std)
    sleep_stage_mean = np.array(sleep_stage_mean)
    sleep_stage_std = np.array(sleep_stage_std)

    def plot_graph(include_std, include_crossings, filename_suffix):
        plt.figure(figsize=(14, 7))
        plt.xlabel('Time to sleep onset (min)', fontsize=14, fontweight='bold')
        plt.ylabel('Probability', fontsize=14, fontweight='bold')
        xticks = np.arange(0, len(y_pred_mean), 20 // window_for_averaging)
        plt.xticks(xticks, timegrid[xticks * window_for_averaging], fontsize=12, fontweight='bold')
        plt.yticks(fontsize=12, fontweight='bold')

        plt.plot(y_pred_mean, label='Predicted Probability' if ifsubjecttest else 'Mean Predicted Probability', color='blue')
        if include_std:
            plt.fill_between(np.arange(len(y_pred_mean)), y_pred_mean - y_pred_std, y_pred_mean + y_pred_std, color='blue', alpha=0.1)

        plt.plot(sleep_stage_mean, label='Sleep Stages' if ifsubjecttest else 'Mean Sleep Stages', color='purple', linestyle='--', linewidth=2.5)
        if include_std:
            plt.fill_between(np.arange(len(sleep_stage_mean)), sleep_stage_mean - sleep_stage_std, sleep_stage_mean + sleep_stage_std, color='purple', alpha=0.1)

        plt.hlines(0.5, linestyle='--', color='r', xmin=0, xmax=len(y_pred_mean), label='Threshold for classifying as pre-sleep onset', linewidth=2.5)
        if include_crossings and change_points_times is not None:
            plt.vlines(change_points_times, ymin=-0.1, ymax=1.1, color='seagreen', label='Detected change points', linewidth=2.5, linestyle='--')

        if awake_window:
            awake_window_probas = int((awake_window - rnn_window) * 10 // window_for_averaging)
            plt.axvspan(awake_window_probas, len(y_pred_mean), facecolor='g', alpha=0.2, label='Awake window')
        if pre_sleep_window:
            pre_sleep_window_probas = int((pre_sleep_window - rnn_window) * 10 // window_for_averaging)
            plt.axvspan(pre_sleep_window_probas, 0, facecolor='r', alpha=0.2, label='Pre-sleep window')

        plt.ylim(-0.1, 1.1)
        plt.legend(loc='upper right', fontsize=14)
        filename = f"sleep_probabilities_{filename_suffix}_averaged_over_{window_for_averaging}.jpg"
        if ifsaveplots:
            if output_path:
                filename = os.path.join(output_path, filename)
            plt.savefig(filename, format='jpg', dpi=200)
        if ifplot:
            plt.show()
        plt.close()

    plot_graph(False, False, "without_std")
    plot_graph(True, False, "with_std")
    plot_graph(False, True, "without_std_with_crossings")
    plot_graph(True, True, "with_std_with_crossings")



def calculate_metrics(y_pred, y_true, timegrid, indices):
    correct = y_true == y_pred
    accuracies = [np.mean(correct[indices[value]]) if not np.isnan(indices[value]).any() else np.nan for value in timegrid]

    prediction_dict_by_time = {value: y_pred[indices[value]] if indices[value] is not np.nan else np.nan for value in timegrid}
    true_values_dict_by_time = {value: y_true[indices[value]] if indices[value] is not np.nan else np.nan for value in timegrid}

    tp = {value: np.sum(np.logical_and(prediction_dict_by_time[value] == 1, true_values_dict_by_time[value] == 1)) if indices[value] is not np.nan else np.nan for value in timegrid}
    tn = {value: np.sum(np.logical_and(prediction_dict_by_time[value] == 0, true_values_dict_by_time[value] == 0)) if indices[value] is not np.nan else np.nan for value in timegrid}
    fp = {value: np.sum(np.logical_and(prediction_dict_by_time[value] == 1, true_values_dict_by_time[value] == 0)) if indices[value] is not np.nan else np.nan for value in timegrid}
    fn = {value: np.sum(np.logical_and(prediction_dict_by_time[value] == 0, true_values_dict_by_time[value] == 1)) if indices[value] is not np.nan else np.nan for value in timegrid}

    recall = [tp[value] / (tp[value] + fn[value]) for value in timegrid]
    precision = [tp[value] / (tp[value] + fp[value]) for value in timegrid]
    f1 = [2 * (precision[i] * recall[i]) / (precision[i] + recall[i]) if precision[i] + recall[i] != 0 else np.nan for i in range(len(precision))]

    return accuracies, recall, precision, f1

def plot_metric(metric_values, metric_name, color, timegrid, window_for_averaging, rnn_window,
                pre_sleep_window=None, awake_window=None, ifsaveplots=False, output_path=None,
                ifplot = False):

    plt.figure(figsize=(20,10))
    plt.xlabel('Minutes', fontsize=14)
    xticks = np.arange(0, len(metric_values), 10//window_for_averaging)
    plt.xticks(xticks, timegrid[xticks*window_for_averaging], fontsize=12)
    plt.yticks(fontsize=12)

    plt.plot(metric_values, label=metric_name, color=color)
    #plt.hlines(0.5, linestyle='--', color='r', label='Threshold for classifying as pre-sleep onset', xmin=0, xmax=len(metric_values))

    if awake_window is not None:
        awake_window_probas = int((awake_window - rnn_window)* 10//window_for_averaging)
        plt.axvspan(awake_window_probas, len(metric_values), facecolor='g', alpha=0.2, label='Awake window')

    if pre_sleep_window is not None:
        pre_sleep_window_probas = int((pre_sleep_window - rnn_window) * 10//window_for_averaging)
        plt.axvspan(pre_sleep_window_probas, 0, facecolor='r', alpha=0.2, label='Pre-sleep window')

    plt.ylim(-0.1, 1.1)
    plt.legend(loc='upper right', fontsize=12)
    if ifsaveplots:
        filename = f"{metric_name}_metric_averaged_over_{window_for_averaging}.jpg"
        if output_path:
            filename = os.path.join(output_path, filename)
        plt.savefig(filename, format='jpg', dpi=300)

    if ifplot:
        plt.show()
    plt.close()


def plot_metrics(accuracies, #recall, precision, f1,
                 timegrid, window_for_averaging, rnn_window,
                 pre_sleep_window=None, awake_window=None,
                 ifsaveplots=False, output_path=None, ifplot = False):

    plot_metric(accuracies, 'Accuracy', 'blue', timegrid, window_for_averaging, rnn_window,
                pre_sleep_window, awake_window, ifsaveplots, output_path, ifplot = ifplot)

    #plot_metric(recall, 'Recall', 'green', timegrid, window_for_averaging, rnn_window,
                #pre_sleep_window, awake_window, ifsaveplots, output_path, ifplot = ifplot)

    #plot_metric(precision, 'Precision', 'purple', timegrid, window_for_averaging, rnn_window,
                #pre_sleep_window, awake_window, ifsaveplots, output_path, ifplot = ifplot)

    #plot_metric(f1, 'F1', 'red', timegrid, window_for_averaging, rnn_window,
                #pre_sleep_window, awake_window, ifsaveplots, output_path, ifplot = ifplot)





### Define functions for probability change point analysis

In [27]:
def nan_convolve(array, window, mode='valid'):
    if mode != 'valid':
        raise ValueError("Only 'valid' mode is currently supported")

    output = np.zeros(len(array) - len(window) + 1)
    for i in range(len(output)):
        current_slice = array[i: i + len(window)]
        valid_data = current_slice[~np.isnan(current_slice)]
        valid_window = window[:len(valid_data)]
        output[i] = np.sum(valid_data * valid_window)
    return output

In [28]:
def threshold_consistency_change_point(data, times, threshold = 0.5, consistency_length=None, window_size=20, proportion=0.7, stay_above_proportion=0.8):
    #assert len(data) == len(times), "Data and times must have the same length"

    if type(data) == list:
        data = np.array(data)
    if type(times) == list:
        times = np.array(times)


    data = data[::-1]

    above = data > threshold


    change_points_consistent = np.array([])  # Initialize here
    # reverse data

    #print(data)
    #print(times)

    # Check for consistent crossing
    if consistency_length:
        consistent_above = nan_convolve(above, np.ones(consistency_length), 'valid') == consistency_length
        change_points_consistent = np.where(consistent_above)[0] + consistency_length - 1

    # Check for proportion in a running window
    if not consistency_length or (window_size and proportion):
        within_window = nan_convolve(above, np.ones(window_size), 'valid') / window_size
        change_points_window = np.where(within_window > proportion)[0] + window_size - 1
        change_points_indices = np.sort(np.unique(np.concatenate([change_points_consistent, change_points_window])))
    else:
        change_points_indices = change_points_consistent

    old_indices = change_points_indices.copy()

    final_change_points = np.array([])  # Initialize here
    last_index = None


    # change to integer
    change_points_indices = change_points_indices.astype(int)
    #print("change_points_indices: ", change_points_indices)

    for index in change_points_indices:
        #print("index: ", index)
        preceding_data = data[:index]
        # calculate how many values are above the threshold (>threshold) while omitting nans
        percentage =  np.nansum(preceding_data > threshold)/ np.sum(~np.isnan(preceding_data))
       #print("percentage: ", percentage)
        if percentage > stay_above_proportion:
           #print('Delete index: ', index)
            # delete the last entry from the final change points
            if len(final_change_points) > 0:
                final_change_points = np.delete(final_change_points, -1)

            change_points_indices = np.delete(change_points_indices, np.where(change_points_indices == index)[0])
            #print("change_points_indices after deletion: ", change_points_indices)
            # add index to final change points

            final_change_points = np.append(final_change_points, index)
            #print("final_change_points after deletion: ", final_change_points)
        else:
            #print('index that broke consistency: ', index)
            # add the remaining change points (change_points_indices) to the final change points
            final_change_points = np.concatenate([final_change_points, change_points_indices])
            #print("final_change_points after concatenation: ", final_change_points)
            break


    if len(final_change_points) == 0 and len(old_indices) > 0:
        final_change_points = np.array([old_indices[-1]])

    # Ensure all indices are integers
    final_change_points = final_change_points.astype(int)

    # Convert indices to associated times
    change_points_times = times[final_change_points].tolist()

    # Check if the last timepoint is the largest one in the original time grid
    if final_change_points is not None and len(final_change_points) > 0:
        # the last index of data that is not nan
        last_index = np.where(~np.isnan(data))[0][-1]
        if final_change_points[0] == last_index:
            return np.array([]), np.array([])
        elif change_points_times[0] > 28:
           return np.array([]), np.array([])

    del change_points_indices, change_points_consistent, change_points_window

    return final_change_points, change_points_times







### Define the loop for probability analysis

In [29]:
def check_accuracy_timeline(y_pred, y_true, timeline, sleep_stage, window_for_averaging=1,
                            if_proba=False, classes=None, pre_sleep_window=None,
                            awake_window=None, rnn_window=None, ifsaveplots=False, output_path=None,
                            ifplot = False, ifsubjecttest = False, ifcrossings = False):
    y_pred, y_true, timeline, sleep_stage = preprocess_data(y_pred, y_true, timeline, sleep_stage)

    indices, unique_values_sorted = get_indices_for_unique_values(timeline)
    indices, timegrid = update_indices_with_missing_values(indices, rnn_window)

    if if_proba:
        if classes is not None:
            class_idx = np.where(classes != 'Awake')[0][0]


            y_pred = y_pred[:,class_idx]

        y_pred_mean, y_pred_std = sort_predictions_by_time(y_pred, indices, timegrid)
        sleep_stage_mean, sleep_stage_std = sort_predictions_by_time(sleep_stage, indices, timegrid)


        if window_for_averaging > 1:
            y_pred_mean = downsample_data(y_pred_mean, window_for_averaging)
            y_pred_std = downsample_data(y_pred_std, window_for_averaging)
            sleep_stage_mean= downsample_data(sleep_stage_mean, window_for_averaging)
            sleep_stage_std = downsample_data(sleep_stage_std, window_for_averaging)
            downsampled_timegrid = downsample_data(timegrid, window_for_averaging)


        change_points_times = np.array([])
        final_change_points = np.array([])

        if ifcrossings:
            window_size = int(20/window_for_averaging)
            if window_for_averaging > 1:
                dict_pred_time = {downsampled_timegrid[i]: y_pred_mean[i] for i in range(len(y_pred_mean))}
                # sort in descending order according to keys
                sorted_dict_pred_time = dict(sorted(dict_pred_time.items(), reverse=True))
                # get the values of sorted dictionary
                y_probas_sorted = list(sorted_dict_pred_time.values())


                final_change_points, change_points_times = threshold_consistency_change_point(data = y_probas_sorted,
                                                                        times = downsampled_timegrid,
                                                                        threshold=0.5,
                                                                        window_size=window_size, proportion=0.8, stay_above_proportion=0.6)
            else:
                dict_pred_time = {timegrid[i]: y_pred_mean[i] for i in range(len(y_pred_mean))}
                # sort in descending order according to keys
                sorted_dict_pred_time = dict(sorted(dict_pred_time.items(), reverse=True))
                # get the values of sorted dictionary
                y_probas_sorted = list(sorted_dict_pred_time.values())

                final_change_points, change_points_times = threshold_consistency_change_point(data = y_probas_sorted,
                                                                        times = timegrid,
                                                                        threshold=0.5,
                                                                        window_size=window_size, proportion=0.8, stay_above_proportion=0.6)

            num_change_points = len(final_change_points)
            if num_change_points == 1:
                time_of_prediction = change_points_times[0]
            else:
                time_of_prediction = np.nan


        plot_data(y_pred_mean, y_pred_std, sleep_stage_mean, sleep_stage_std,
                  timegrid, window_for_averaging, rnn_window,
                  pre_sleep_window, awake_window, ifsaveplots,
                  output_path, ifplot = ifplot, ifsubjecttest = ifsubjecttest,
                   change_points_times = final_change_points)

        if window_for_averaging > 1:
            if ifcrossings:

                del  dict_pred_time, sorted_dict_pred_time, y_probas_sorted, final_change_points, change_points_times

                return num_change_points, time_of_prediction
            else:

                dict_pred_time = {downsampled_timegrid[i]: y_pred_mean[i] for i in range(len(y_pred_mean))}

                del downsampled_timegrid

                return dict_pred_time
        elif ifcrossings:
            del dict_pred_time, sorted_dict_pred_time, y_probas_sorted, final_change_points, change_points_times
            return num_change_points, time_of_prediction
        else:
            del timegrid
            return y_pred_mean

    else:
        accuracies, recall, precision, f1 = calculate_metrics(y_pred, y_true, timegrid, indices)
        if window_for_averaging > 1:
            accuracies = downsample_data(accuracies, window_for_averaging)
            #recall = downsample_data(recall, window_for_averaging)
            #precision = downsample_data(precision, window_for_averaging)
            #f1 = downsample_data(f1, window_for_averaging)


        plot_metrics(accuracies, #recall, precision, f1,
                     timegrid, window_for_averaging, rnn_window,
                     pre_sleep_window, awake_window, ifsaveplots,
                     output_path, ifplot = ifplot)

        return accuracies, recall, precision, f1



### Define the training and evaluation functions for regression

In [30]:
import torch
from sklearn.metrics import mean_squared_error, r2_score

def train_regression(model, train_loader, num_epochs, criterion,
                     optimizer, device, ifplot=False, ifvalidation = True, val_loader = None):

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))

        # Train the model
        model.train()
        running_train_loss = 0.0
        train_predictions, train_actuals = [], []
        for i, (inputs, labels_and_endpoints) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels_and_endpoints[:, 1]
            labels= labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels.view(-1, 1))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
            outputs_detached = outputs.detach()
            train_predictions.extend(outputs_detached.cpu().numpy().flatten())
            train_actuals.extend(labels.cpu().numpy())

        avg_train_loss = running_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)



        if ifvalidation:
            # Validate the model
            model.eval()
            running_val_loss = 0.0
            val_predictions, val_actuals = [], []
            with torch.no_grad():
                for inputs, labels_and_endpoints in val_loader:
                    inputs = inputs.to(device)
                    labels = labels_and_endpoints[:, 1]
                    labels = labels.to(device)

                    outputs = model(inputs)
                    running_val_loss += criterion(outputs, labels.view(-1, 1)).item()
                    val_predictions.extend(outputs.cpu().numpy().flatten())
                    val_actuals.extend(labels.cpu().numpy())



                avg_val_loss = running_val_loss / len(val_loader)
                val_losses.append(avg_val_loss)

        # Calculate metrics
        if ifvalidation:
            mse = mean_squared_error(val_actuals, val_predictions)
            r2 = r2_score(val_actuals, val_predictions)
            mae = mean_absolute_error(val_actuals, val_predictions)
        else:
            mse = mean_squared_error(train_actuals, train_predictions)
            r2 = r2_score(train_actuals, train_predictions)
            mae = mean_absolute_error(train_actuals, train_predictions)

        if ifplot:
            print('-'*100)
            if ifvalidation:
                print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')
            else:
                print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}')
            print(f'Mean Squared Error: {mse}, R2 Score: {r2}')
            print(f'Mean Absolute Error: {mae}')
    if ifvalidation:
        del train_predictions, train_actuals
        return  val_predictions, val_actuals, train_losses, val_losses, model, optimizer
    else:
        return  train_predictions, train_actuals, train_losses, model, optimizer


# Test function
def test_regression(model, test_loader, device, criterion, ifplot=False):
    model.eval()
    predictions, actuals = [], []

    with torch.no_grad():
        test_loss = 0
        for inputs, labels_and_endpoints in test_loader:
            inputs = inputs.to(device)
            labels = labels_and_endpoints[:, 1].to(device)

            outputs = model(inputs)  # these are outputs now
            test_loss += criterion(outputs, labels.view(-1, 1)).item()

            predictions.extend(outputs.cpu().numpy().flatten())
            actuals.extend(labels.cpu().numpy())
        if ifplot:
            print(f'Test Loss: {test_loss/len(test_loader)}')

    # Calculate metrics
    mse = mean_squared_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    if ifplot:
        print('Mean Squared Error:', mse)
        print('R2 Score:', r2)
        print('Mean Absolute Error:', mae)
    del mse, r2, mae
    return predictions, actuals

In [31]:
def evaluate_regression(predictions, y_test, ifsaveplots = False, savepath = None, ifplot = False, iftest = False):

    # check if the predictions are in the correct format

    predictions = np.array(predictions)

    # check if the y_test is in the correct format

    y_test = np.array(y_test)

    errors = abs(y_test - predictions)
    mape = np.where(y_test != 0, 100 * (errors / y_test), 0)
    accuracy = 100 - np.mean(mape)
    mse = sklearn.metrics.mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = sklearn.metrics.r2_score(y_test, predictions)
    squared_errors = np.square(y_test - predictions)
    weights = 1.0 / (abs(y_test) + 0.1)
    custom_mse = np.mean(weights * squared_errors)

    if ifplot:

        print('Mean Absolute Error:', round(np.mean(errors), 2))
        print('Accuracy:', round(accuracy, 2), '%.')
        print('Mean Squared Error:', round(mse, 2))
        print('Root Mean Squared Error:', round(rmse, 2))
        print('R2:', round(r2, 2))
        print('Custom MSE:', round(custom_mse, 2))




    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=1)
    plt.scatter(y_test, predictions, alpha=0.2)
    plt.xlabel('Actual time to sleep onset')
    plt.ylabel('Predicted time to sleep onset')
    plt.title('Actual vs Predicted time to sleep onset')
    if ifsaveplots:
        if iftest:
            plt.savefig(os.path.join(savepath, 'test_actual_vs_predicted.png'))
        else:
            plt.savefig(os.path.join(savepath, 'crossval_actual_vs_predicted.png'))
    if ifplot:
        plt.show()
    plt.close()

    residuals = y_test - predictions
    plt.scatter(y_test, residuals, alpha=0.2)
    plt.plot([y_test.min(), y_test.max()], [0, 0], 'k--', lw=1)
    plt.xlabel('Actual time to sleep onset')
    plt.ylabel('Residuals')
    plt.title('Actual vs Residuals for predicted time to sleep onset')
    if ifsaveplots:
        if iftest:
            plt.savefig(os.path.join(savepath, 'test_actual_vs_residuals.png'))
        else:
            plt.savefig(os.path.join(savepath, 'crossval_actual_vs_residuals.png'))
    if ifplot:
        plt.show()
    plt.close()

    plt.hist(residuals)
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.title('Residuals distribution')
    if ifsaveplots:
        if iftest:
            plt.savefig(os.path.join(savepath, 'test_residuals_distribution.png'))
        else:
            plt.savefig(os.path.join(savepath, 'crossval_residuals_distribution.png'))
    if ifplot:
        plt.show()
    plt.close()

    # for each actual, plot the mean residual
    # round y_test to 0.1 decimal place
    y_test = np.round(y_test, 1)

    predictions_residuals_dict = {}
    for i in range(len(y_test)):
        if y_test[i] not in predictions_residuals_dict:
            predictions_residuals_dict[y_test[i]] = []
        predictions_residuals_dict[y_test[i]].append(residuals[i])

    # sort it in ascending order of keys
    predictions_residuals_dict = dict(sorted(predictions_residuals_dict.items()))


    actual_predictions_dict = {}
    for i in range(len(y_test)):
        if y_test[i] not in actual_predictions_dict:
            actual_predictions_dict[y_test[i]] = []
        actual_predictions_dict[y_test[i]].append(predictions[i])

    # sort it in ascending order of keys
    actual_predictions_dict = dict(sorted(actual_predictions_dict.items()))


    predictions_residuals_mean_dict = {}
    for key in predictions_residuals_dict:
        predictions_residuals_mean_dict[key] = np.mean(predictions_residuals_dict[key])

    # sort it in ascending order of keys
    predictions_residuals_mean_dict = dict(sorted(predictions_residuals_mean_dict.items()))


    residuals_std_dict = {}
    for key in predictions_residuals_dict:
        residuals_std_dict[key] = np.std(predictions_residuals_dict[key])
    # sort it in ascending order of keys
    residuals_std_dict = dict(sorted(residuals_std_dict.items()))


    actuals_predictions_mean_dict = {}
    for key in actual_predictions_dict:
        actuals_predictions_mean_dict[key] = np.mean(actual_predictions_dict[key])

    # sort it in ascending order of keys
    actuals_predictions_mean_dict = dict(sorted(actuals_predictions_mean_dict.items()))

    predictions_std_dict = {}
    for key in actual_predictions_dict:
        predictions_std_dict[key] = np.std(actual_predictions_dict[key])
    # sort it in ascending order of keys
    predictions_std_dict = dict(sorted(predictions_std_dict.items()))

    plt.plot(list(predictions_residuals_mean_dict.keys()), list(predictions_residuals_mean_dict.values()))
    # create std shading
    plt.fill_between(list(predictions_residuals_mean_dict.keys()),
                        np.array(list(predictions_residuals_mean_dict.values())) - np.array(list(residuals_std_dict.values())),
                        np.array(list(predictions_residuals_mean_dict.values())) + np.array(list(residuals_std_dict.values())),
                        alpha=0.2)
    # plot ideal line (x = 0)
    plt.plot([y_test.min(), y_test.max()], [0, 0], 'k--', lw=1, color='red')
    plt.legend(['Mean of residuals', 'STD of residuals', 'Ideal residuals line'])
    plt.xlabel('Actual time to sleep onset')
    plt.ylabel('Residuals for this actual')
    plt.title('Mean residual of predictions for each actual time to sleep onset')
    if ifsaveplots:
        if iftest:
            plt.savefig(os.path.join(savepath, 'test_mean_residuals_across_time.png'))
        else:
            plt.savefig(os.path.join(savepath, 'crossval_mean_residuals_across_time.png'))
    if ifplot:
        plt.show()
    plt.close()

    plt.plot(list(actuals_predictions_mean_dict.keys()), list(actuals_predictions_mean_dict.values()))
    # create std shading
    plt.fill_between(list(actuals_predictions_mean_dict.keys()),
                        np.array(list(actuals_predictions_mean_dict.values())) - np.array(list(predictions_std_dict.values())),
                        np.array(list(actuals_predictions_mean_dict.values())) + np.array(list(predictions_std_dict.values())),
                        alpha=0.2)
    # plot ideal line (x = y)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=1, color='red')
    plt.legend(['Mean of predictions', 'STD of predictions', 'Ideal actual = prediction line (x = y)'])
    plt.xlabel('Actual time to sleep onset')
    plt.ylabel('Mean prediction for this actual')
    plt.title('Mean prediction for each actual time to sleep onset')
    if ifsaveplots:
        if iftest:
            plt.savefig(os.path.join(savepath, 'test_mean_predictions_across_time.png'))
        else:
             plt.savefig(os.path.join(savepath, 'crossval_mean_predictions_across_time.png'))
    if ifplot:
        plt.show()
    plt.close()

    test_metrics = {'MAE': round(np.mean(errors), 2), 'Accuracy': round(accuracy, 2),
    'MSE': round(mse, 2), 'RMSE': round(rmse, 2), 'R2': round(r2, 2), 'custom_MSE': round(custom_mse, 2)}


    del predictions, predictions_residuals_dict
    del actual_predictions_dict, predictions_residuals_mean_dict
    del actuals_predictions_mean_dict, predictions_std_dict, residuals, residuals_std_dict, y_test
    del mse, rmse, r2, custom_mse

    return test_metrics

### Define functions for assessing the model's performance on a single participant

In [32]:
def select_random_participants(df, n_participants):
    """Selects a random sample of participants from the dataset.

    Args:
        df (pandas.DataFrame): The dataset.
        n_participants (int): The number of participants to select.

    Returns:
        pandas.DataFrame: A random sample of participants.
    """
    # Select a random sample of participants
    participants = df['Sbj_ID'].unique()
    np.random.seed(42)
    selected_participants = np.random.choice(participants, n_participants, replace=False)

    for participant in selected_participants:
        # check how many nan values there are in the dataset for this participant
        n_nans = df[df['Sbj_ID'] == participant].isna().sum().sum()
        # if there are more than 20 nan values, select another participant
        while n_nans > 50:
            selected_participants = np.delete(selected_participants, np.where(selected_participants == participant))
            new_participant = np.random.choice(participants, 1, replace=False)
            # check if the new participant is already in the list
            if new_participant not in selected_participants:
                selected_participants = np.append(selected_participants, new_participant)
            else:
                continue
            participant = selected_participants[-1]
            n_nans = df[df['Sbj_ID'] == participant].isna().sum().sum()

    # Filter the dataset
    df = df[df['Sbj_ID'].isin(selected_participants)].copy()

    return df, selected_participants

In [33]:
def test_on_random_participants(data, model, criterion, random_seed, ifplot,
                                device, output_path_new, index, convergence_point,
                                awake_window, pre_sleep_window, ifsaveplots = True,
                                window_size_minutes = 1, ifmissing = True,
                                method = None, resampling_method = None, filling_limit = 0):

    """
    Test the model on a random subset of participants.
    """
    # create a new folder for the results
    output_path_new = os.path.join(output_path_new, 'random_participants')

    if not os.path.exists(output_path_new):
        os.makedirs(output_path_new)

    # intialise empty dataframe to store results
    df_all = pd.DataFrame()

    participant_dataloaders = {}

    for i, sbj in enumerate(data['Sbj_ID'].unique()):

        print('Participant {} of {}'.format(i + 1, len(data['Sbj_ID'].unique())))

        # create a folder for each participant
        output_path_participant = os.path.join(output_path_new, str(sbj))
        if ifsaveplots:
            if not os.path.exists(output_path_participant):
                os.makedirs(output_path_participant)

        participant_data = data[data['Sbj_ID'] == sbj].copy()

        test_sequences_sbj, test_labels_sbj, test_end_points_sbj, test_sleep_stages_sbj, index, label = prepare_classification_data(mydata_train = participant_data,
                                                                                                        sleep_onset_threshold = convergence_point,
                                                                                                        awake_threshold = convergence_point,
                                                                                                        method = method, filling_limit = filling_limit,
                                                                                                        window_size_minutes = window_size_minutes,
                                                                                                        ifmissing = ifmissing, random_seed = random_seed,
                                                                                                        resampling_method = None)

        # Get the tuple of test labels and test end points
        test_labels_and_end_points_sbj = [(test_labels_sbj[i], test_end_points_sbj[i]) for i in range(len(test_labels_sbj))]

        # Create the test dataset and dataloader with labels and end points (for the regression head)
        test_dataset_sbj = SleepDataset(test_sequences_sbj, test_labels_and_end_points_sbj)
        test_loader_sbj = DataLoader(test_dataset_sbj, batch_size=1, shuffle=False)

        predictions, actuals, probas = test_classification(model, test_loader_sbj, device, criterion, index, threshold=0.5, ifprobabilities=True)
        timeline = test_end_points_sbj #participant_data['Old_label']
        participant_dataloaders[sbj] = test_loader_sbj

        # get the accuracy of each timepoint
        if ifplot:
            print('Predict for participant: ', sbj)

            print('--------------------------------------Evaluate classificaiton ----------------------------------------')

        test_metrics = evaluate_classification(actuals, predictions, probas, index = index, iftest=True, columns = label,
                                ifplot=ifplot, output_path=output_path_participant, ifsaveplots = ifsaveplots, classes = None, ifvocal = False)
        test_metrics['Sbj_ID'] = sbj


        if ifplot:
                print('------------------------------------------- No averaging ----------------------------------------')
                print('Accuracy')
        _, _, _, _ = check_accuracy_timeline(predictions, test_labels_sbj, timeline,
                                            test_sleep_stages_sbj, window_for_averaging= 1, awake_window = awake_window,
                                            pre_sleep_window = pre_sleep_window, rnn_window = window_size_minutes,
                                            ifsaveplots = ifsaveplots, output_path= output_path_participant, ifplot = ifplot)
        if ifplot:
            print('Probability of sleep as predicted by the model')
        num_crossings, time_of_prediction = check_accuracy_timeline(probas, test_labels_sbj, timeline,
                                                test_sleep_stages_sbj, window_for_averaging= 1, if_proba = True,
                                                classes = index, awake_window = awake_window, pre_sleep_window = pre_sleep_window,
                                                rnn_window = window_size_minutes, ifsaveplots = ifsaveplots, output_path= output_path_participant,
                                                ifplot = ifplot, ifsubjecttest = True, ifcrossings = True)
        test_metrics['num_crossings'] = num_crossings
        test_metrics['time_of_prediction'] = time_of_prediction

        if ifplot:
            print('-------------------------------Averaging of accuracy over 30 seconds---------------------------------')
            print('Accuracy')
        _, _, _, _ = check_accuracy_timeline(predictions, test_labels_sbj, timeline,
                                            test_sleep_stages_sbj, window_for_averaging= 5,
                                            awake_window = awake_window, pre_sleep_window = pre_sleep_window,
                                            rnn_window = window_size_minutes, ifsaveplots = ifsaveplots,
                                            output_path= output_path_participant, ifplot = ifplot)
        if ifplot:
            print('Probability of sleep as predicted by the model')
        num_crossings_smoothed, time_of_prediction_smoothed = check_accuracy_timeline(probas, test_labels_sbj, timeline,
                                                test_sleep_stages_sbj, window_for_averaging= 5, if_proba = True,
                                                classes = index, awake_window = awake_window, pre_sleep_window = pre_sleep_window,
                                                rnn_window = window_size_minutes, ifsaveplots = ifsaveplots, output_path= output_path_participant,
                                                ifplot = ifplot, ifsubjecttest = True, ifcrossings = True)

        test_metrics['num_crossings_smoothed'] = num_crossings_smoothed
        test_metrics['time_of_prediction_smoothed'] = time_of_prediction_smoothed

        del test_dataset_sbj, test_loader_sbj, test_labels_and_end_points_sbj, test_sequences_sbj, test_labels_sbj, test_end_points_sbj, test_sleep_stages_sbj
        del predictions, actuals, probas


        next_index = len(df_all)
        df = pd.DataFrame(test_metrics, index=[next_index])
        df_all = pd.concat([df_all, df])

        del test_metrics

    df_all.to_csv(os.path.join(output_path_new, 'classification_results_random_participants.csv'))

    # calculate how many participants have only one crossing in df_all['num_crossings'] out of all the participants
    num_participants_one_crossing = len(df_all[df_all['num_crossings'] == 1])
    num_participants = len(df_all)
    one_crossing_percentage = num_participants_one_crossing/num_participants * 100
    mean_time_of_prediction = np.mean(df_all['time_of_prediction'][df_all['num_crossings'] == 1])
    std_time_of_prediction = np.std(df_all['time_of_prediction'][df_all['num_crossings'] == 1])


    num_participants_one_crossing_smoothed = len(df_all[df_all['num_crossings_smoothed'] == 1])
    one_crossing_percentage_smoothed = num_participants_one_crossing_smoothed/num_participants * 100
    mean_time_of_prediction_smoothed = np.mean(df_all['time_of_prediction_smoothed'][df_all['num_crossings_smoothed'] == 1])
    std_time_of_prediction_smoothed = np.std(df_all['time_of_prediction_smoothed'][df_all['num_crossings_smoothed'] == 1])


    print(f'Percentage of participants with only one crossing: {one_crossing_percentage}%')
    print(f'Mean time of prediction for participants with only one crossing: {mean_time_of_prediction} minutes')
    print(f'STD of time of prediction for participants with only one crossing: {std_time_of_prediction} minutes')


    print(f'Percentage of participants with only one crossing (smoothed): {one_crossing_percentage_smoothed}%')
    print(f'Mean time of prediction for participants with only one crossing (smoothed): {mean_time_of_prediction_smoothed} minutes')
    print(f'STD of time of prediction for participants with only one crossing (smoothed): {std_time_of_prediction_smoothed} minutes')

    output_dict = {'one_crossing_percentage': one_crossing_percentage, 'mean_time_of_prediction': mean_time_of_prediction,
                    'std_time_of_prediction': std_time_of_prediction, 'one_crossing_percentage_smoothed': one_crossing_percentage_smoothed,
                    'mean_time_of_prediction_smoothed': mean_time_of_prediction_smoothed, 'std_time_of_prediction_smoothed': std_time_of_prediction_smoothed}

    del one_crossing_percentage, mean_time_of_prediction, std_time_of_prediction, one_crossing_percentage_smoothed, mean_time_of_prediction_smoothed, std_time_of_prediction_smoothed

    return participant_dataloaders, output_dict

In [34]:
def test_reg_on_random_participants(participant_dataloaders, model, criterion, device, random_seed, ifplot, output_path_new, index):

    """
    Test the model on a random subset of participants.
    """
    # Set random seed

    # create a new folder for the results
    output_path_new = os.path.join(output_path_new, 'random_participants')
    if not os.path.exists(output_path_new):
        os.makedirs(output_path_new)

    # intialise empty dataframe to store results
    df_all = pd.DataFrame()


    for sbj in list(participant_dataloaders.keys()):
        # create a folder for each participant

        output_path_participant = os.path.join(output_path_new, str(sbj))
        if not os.path.exists(output_path_participant):
            os.makedirs(output_path_participant)

        test_dataloader = participant_dataloaders[sbj]

        predictions, actuals = test_regression(model, test_dataloader, device, criterion)

        # get the accuracy of each timepoint
        if ifplot:
            print('Predict for participant: ', sbj)

            print('--------------------------------------Evaluate regression ----------------------------------------')

        test_metrics = evaluate_regression(predictions, actuals, iftest=True,
                                ifplot=ifplot, savepath =output_path_participant, ifsaveplots= True)


        del test_dataloader, predictions, actuals
        df = pd.DataFrame(test_metrics, index = [sbj])
        df_all = pd.concat([df_all, df])
        del test_metrics

    df_all.to_csv(os.path.join(output_path_new, 'regression_results_random_participants.csv'))



In [35]:

def train_test_split_sbjs(mydata_no_nan, test_size = 0.2, random_seed = 42):

    # assert that the data has no NaNs
    assert mydata_no_nan.isnull().sum().sum() == 0, 'There are NaNs in the data'

    # get the unique subjects
    unique_subjects = mydata_no_nan['Sbj_ID'].unique()

    # define the train size
    train_size = 1 - test_size

    # define the number of subjects in the test set

    test_size_subjects = int(test_size * len(unique_subjects))

    # Select the unique subjects for the test set

    test_subjects = random.sample(list(unique_subjects), test_size_subjects)

    # Select the unique subjects for the training set
    train_subjects = [x for x in unique_subjects if x not in test_subjects]

    # Create the test and training sets
    test_set = mydata_no_nan[mydata_no_nan['Sbj_ID'].isin(test_subjects)]
    train_set = mydata_no_nan[mydata_no_nan['Sbj_ID'].isin(train_subjects)]

    # Create X and y from dataframes to use in scikit-learn (drop Label, SleepStage, Sbj_ID and ifCleanOnset columns)

    X_train = train_set.drop(['Label', 'SleepStage', 'Sbj_ID', 'ifCleanOnset', 'Time2Sleep'], axis=1)
    y_train = train_set['Label']
    X_test = test_set.drop(['Label', 'SleepStage', 'Sbj_ID', 'ifCleanOnset', 'Time2Sleep'], axis=1)
    y_test = test_set['Label']


    # Create list of sbj_IDs for each sample in the training and test sets
    sbj_train_set = train_set['Sbj_ID'].copy()
    sbj_test_set = test_set['Sbj_ID'].copy()

    # Turn the sbj_IDs into a list
    sbj_train_set = sbj_train_set.tolist()
    sbj_test_set = sbj_test_set.tolist()



    return X_train, y_train, X_test, y_test, sbj_train_set, sbj_test_set

In [36]:
def get_rid_of_nans(probabilities, y_times2sleep = None):
    if np.isnan(probabilities).any():
        for i in range(len(probabilities)):
            if np.isnan(probabilities[i]):
                # omit the corresponding y_times2sleep
                if y_times2sleep is not None:
                    y_times2sleep.pop(i)
                probabilities.pop(i)
    return probabilities, y_times2sleep

### Define the function for schedule learning

In [37]:
def update_thresholds(epoch, initial_sleep_onset_threshold, initial_awake_threshold, awake_delta, sleep_delta, schedule_period=1):
    """
    Updates the thresholds based on a target convergence point.

    Parameters:
    - epoch: current epoch number
    - initial_awake_threshold: initial awake threshold
    - initial_sleep_onset_threshold: initial sleep onset threshold
    - convergence_point: point at which the thresholds should converge
    - total_epochs: total epochs for which the training is scheduled
    - schedule_period: epochs after which thresholds are adjusted. Default is 10 epochs.

    Returns:
    - New awake and sleep_onset thresholds
    """

    # Adjusting both thresholds based on the epoch
    if epoch % schedule_period == 0 :
        initial_awake_threshold -= awake_delta
        initial_sleep_onset_threshold += sleep_delta

    return round(initial_sleep_onset_threshold, 1), round(initial_awake_threshold,1)

### Define helper functions for reducing the length of schedule learning loop

In [38]:
def prepare_classification_data(mydata_train, sleep_onset_threshold,
                                awake_threshold, method, filling_limit, window_size_minutes,
                                ifmissing, random_seed, resampling_method,
                                ):

    mydata_train_class, index, label = regression_to_classification(mydata_train, sleep_onset_threshold, awake_threshold)

    mydata_imputed_class = impute_missing_values(mydata_train_class, method=method, limit=filling_limit)


    train_sequences, train_labels, _, _, _, _, train_end_points, train_sleep_stages = create_sliding_windows(data=mydata_imputed_class, window_size_minutes=window_size_minutes,
                                                                                                        ifmissing=ifmissing, random_seed=random_seed, train_proportion=1,
                                                                                                        if_stratified_sampling=0, resampling_method=resampling_method,
                                                                                                          iftest=0, ifoutput_end_points=1)
    del mydata_imputed_class
    del mydata_train

    return train_sequences, train_labels, train_end_points, train_sleep_stages, index, label

def define_loss_function(ifclassweights, train_labels, device, ifplot = False):
    if ifclassweights:
        class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
        class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
        pos_class_idx = np.where(np.unique(train_labels) == 1)[0][0]
        pos_weight = torch.tensor([class_weights[pos_class_idx]]).to(device)
        if ifplot:
            print('Positive class weights: ', pos_weight)
        return nn.BCEWithLogitsLoss(pos_weight=pos_weight), pos_weight
    else:
        return nn.BCEWithLogitsLoss(), None

def train_and_evaluate_classification_model(model, train_loader, label,  test_labels, test_end_points,
                                            test_sleep_stages,
                                            test_loader, device, criterion,
                                            optimizer, num_epochs, index, ifprobabilities,
                                            ifplot, ifsaveplots, save_directory_epoch,
                                            awake_threshold, sleep_onset_threshold, window_size_minutes):
    if ifprobabilities:
        train_losses, test_losses, model, optimizer, test_predictions, test_actuals, test_prediction_probabilities = train_classification(model = model, train_loader=train_loader,
                                                                                                                                          val_loader=test_loader,
                                                                                                                                          device = device,
                                                                                                                                          criterion=criterion, optimizer=optimizer,
                                                                                                                                          num_epochs=num_epochs, index=index,
                                                                                                                                          ifprobabilities=ifprobabilities,
                                                                                                                                          ifplot = ifplot)

        test_metrics = evaluate_classification(true_labels=test_actuals, predictions=test_predictions,
                                               prediction_probabilities=test_prediction_probabilities,
                                               index=index, columns = label, ifsaveplots=ifsaveplots,
                                               output_path=save_directory_epoch, ifplot=ifplot)

        if ifplot:
                print('------------------------------------------- No averaging ----------------------------------------')
                print('Accuracy')
        accuracies, precision, recall, f1 = check_accuracy_timeline(test_predictions, test_labels, test_end_points,
                                                                    test_sleep_stages, window_for_averaging= 1, awake_window=awake_threshold,
                                                                    pre_sleep_window=sleep_onset_threshold, rnn_window = window_size_minutes,
                                                                    ifsaveplots = ifsaveplots, output_path= save_directory_epoch, ifplot = ifplot)
        if ifplot:
            print('Probability of sleep as predicted by the model')
        y_pred_sorted = check_accuracy_timeline(test_prediction_probabilities, test_labels, test_end_points,
                                                test_sleep_stages, window_for_averaging= 1, if_proba = True,
                                                classes = index, awake_window=awake_threshold, pre_sleep_window=sleep_onset_threshold,
                                                rnn_window = window_size_minutes, ifsaveplots = ifsaveplots, output_path= save_directory_epoch,
                                                ifplot = ifplot)
        if ifplot:
            print('-------------------------------Averaging of accuracy over 30 seconds---------------------------------')
            print('Accuracy')
        accuracies, precision, recall, f1 = check_accuracy_timeline(test_predictions, test_labels, test_end_points,
                                                                    test_sleep_stages, window_for_averaging= 5,
                                                                    awake_window=awake_threshold, pre_sleep_window=sleep_onset_threshold,
                                                                    rnn_window = window_size_minutes, ifsaveplots = ifsaveplots,
                                                                    output_path= save_directory_epoch, ifplot = ifplot)
        if ifplot:
            print('Probability of sleep as predicted by the model')
        y_pred_sorted = check_accuracy_timeline(test_prediction_probabilities, test_labels, test_end_points,
                                                test_sleep_stages, window_for_averaging= 5, if_proba = True,
                                                classes = index,  awake_window=awake_threshold,
                                                pre_sleep_window=sleep_onset_threshold, rnn_window = window_size_minutes,
                                                ifsaveplots = ifsaveplots, output_path= save_directory_epoch, ifplot = ifplot)

        return train_losses, test_losses, model, optimizer, test_predictions, test_actuals, test_prediction_probabilities, test_metrics

    else:
        train_losses, test_losses, model, optimizer, test_predictions, test_actuals, test_prediction_probabilities = train_classification(model = model, train_loader=train_loader,
                                                                                                                                          val_loader=test_loader,
                                                                                                                                          device = device,
                                                                                                                                          criterion=criterion, optimizer=optimizer,
                                                                                                                                          num_epochs=num_epochs, index=index,
                                                                                                                                          ifprobabilities=ifprobabilities,
                                                                                                                                          ifplot = ifplot)

        test_metrics = evaluate_classification(true_labels=test_actuals, predictions=test_predictions,
                                               prediction_probabilities=None,
                                               index=index, columns = label, ifsaveplots=ifsaveplots,
                                               output_path=save_directory_epoch, ifplot=ifplot)

        return train_losses, test_losses, model, optimizer, test_predictions, test_actuals, None, test_metrics

def train_and_evaluate_regression_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs, ifplot, ifsaveplots, output_path):
    test_predictions, test_actuals, train_losses, test_losses, model, optimizer = train_regression(
        model = model, train_loader = train_loader, val_loader = test_loader, num_epochs = num_epochs, criterion = criterion, optimizer =optimizer, device = device, ifplot = ifplot, ifvalidation = True)

    test_metrics = evaluate_regression(test_predictions, test_actuals, ifsaveplots=ifsaveplots, savepath=output_path, ifplot = ifplot)

    return train_losses, test_losses, model, optimizer, test_predictions, test_actuals, test_metrics

In [39]:
def save_models(model_classification, model_regression, save_directory_epoch, sleep_onset_threshold, awake_threshold, window_size_minutes):
    model_path_classification = os.path.join(save_directory_epoch, f"model_classification_presleep_{sleep_onset_threshold}_awake_{awake_threshold}_window_{window_size_minutes}.pt")
    model_regression_path = os.path.join(save_directory_epoch, f"model_regression_presleep_{sleep_onset_threshold}_awake_{awake_threshold}_window_{window_size_minutes}.pt")
    torch.save(model_classification.state_dict(), model_path_classification)
    torch.save(model_regression.state_dict(), model_regression_path)

def plot_losses(train_losses, test_losses, title, ifplot, ifsaveplots, save_directory, filename):
    plt.plot(train_losses, label='Training loss')
    plt.plot(test_losses, label='Validation loss')
    plt.title(title)
    plt.legend()
    if ifplot:
        plt.show()
    if ifsaveplots:
        plt.savefig(os.path.join(save_directory, filename))
    plt.close()

### Define custom MSE loss function

In [40]:
import torch
import torch.nn as nn

class CustomMSELoss(nn.Module):
    def __init__(self, epsilon=0.1):
        super(CustomMSELoss, self).__init__()
        self.epsilon = epsilon  # Small value to prevent division by zero

    def forward(self, predictions, targets):
        # Compute error
        errors = (predictions - targets)**2

        # Compute weights inversely proportional to target values
        weights = 1.0 / (torch.abs(targets) + self.epsilon)

        # Weighted sum of errors
        loss = torch.sum(weights * errors) / torch.sum(weights)
        return loss

# Test the custom loss
predictions = torch.tensor([0.5, 0.2, 0.7])
targets = torch.tensor([0.1, 0.4, 0.8])
criterion = CustomMSELoss()
loss = criterion(predictions, targets)
print(loss)


tensor(0.1099)


### Definte the loop for schedule learning

In [41]:
from sklearn.utils.class_weight import compute_class_weight
import warnings
def training_loop_RNN_regclass_schedule(mydata, convergence_point, total_epochs,  filling_limit, method, ifclassweights, input_size,
                                        num_epochs, hidden_size, num_layers, dropout, l2, lr, weight_decay, batch_size,
                                        device, random_seed = 42, window_size_minutes = 1, ifprobabilitiesanalysis = 0,
                                        if_stratified_sampling = 1, initial_awake_threshold = 30, initial_sleep_onset_threshold = 2,
                                        convergence_sensitivity = 0.1, resampling_method = None, ifplot = False, ifsaveplots = True,
                                        output_path=None, shuffle_train_loader = True, results_saving_dir = None, selected_participants_data = None,
                                        selected_participants = None, ifresetresults = False):

    ################## INITIALISATION OF RESULTS STORAGE ##################
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)

    if results_saving_dir is None:
        results_saving_dir = output_path

    save_directory = output_path

    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # Initialize a DataFrame to store results
    columns = ['convergence_point', 'initial_sleep_onset_threshold','ifconverged', 'pre_sleep_window', 'awake_window', 'pos_class_weight',
                'sliding_window_size', 'Accuracy', 'precision', 'recall', 'f1_weighted',
                'f1_macro', 'f1_pre_sleep', 'f1_awake', 'auc',
                'one_crossing_percentage', 'mean_time_of_prediction',
                'std_time_of_prediction', 'one_crossing_percentage_smoothed',
                'mean_time_of_prediction_smoothed', 'std_time_of_prediction_smoothed',
                'MAE', 'MSE', 'RMSE', 'R2', 'custom_MSE']

    if ifresetresults:
        results_df = pd.DataFrame(columns=columns)
        results_df.to_csv(os.path.join(results_saving_dir, 'results.csv'))

    ################## PREPARE DATA FOR TRAINING AND TESTING ##################
    # Firstly, get the datasets for a classification and regressions problem
    # Create a new column 'old_label' to store the original label
    mydata_bothlabels = mydata.copy()


    mydata_bothlabels['Old_label'] = mydata_bothlabels['Label'].copy()

    # The split we would get after the schedule learning convergence
    mydata_final_split, index, label = regression_to_classification(mydata, convergence_point, convergence_point)

    del mydata
    # Fill in the missing values
    mydata_imputed = impute_missing_values(mydata_final_split, method = method, limit = filling_limit)
    del mydata_final_split
    # Cheсk if the dataset still has any missing values
    ifmissing = mydata_imputed.isnull().sum().sum()

    # Split the data into train, validation and test sets
    _, _, _, _, _, _,  train_subjects, test_subjects, _ = create_sliding_windows(data = mydata_imputed,
                                                                        window_size_minutes=window_size_minutes,
                                                                        ifmissing=ifmissing, random_seed=random_seed,
                                                                        ifoutputsubjects = 1,
                                                                        if_stratified_sampling = if_stratified_sampling,
                                                                        resampling_method = None,
                                                                        train_proportion = 0.8, iftest = 0, ifoutput_end_points= 0)
    del mydata_imputed
    mydata_test = mydata_bothlabels[mydata_bothlabels['Sbj_ID'].isin(test_subjects)]
    mydata_train = mydata_bothlabels[~mydata_bothlabels['Sbj_ID'].isin(test_subjects)]

    ########################## PREPARE THE TESTING DATA  #############################
    # Get the sequences (windows) and labels (categorical 1 or 0 for classification) for the test set,
    # also get the end points time of the sequences (in minutes) and their corresponding sleep stages (at the end of the sequence)
    test_sequences, test_labels, test_end_points, test_sleep_stages, _, _,= prepare_classification_data(mydata_train = mydata_test, sleep_onset_threshold = convergence_point,
                                                                                                        awake_threshold = convergence_point, method = method,
                                                                                                        filling_limit = filling_limit,
                                                                                                        window_size_minutes = window_size_minutes,
                                                                                                        ifmissing = ifmissing, random_seed = random_seed,
                                                                                                        resampling_method = None,
                                                                                                        )

    # Get the tuple of test labels and test end points
    test_labels_and_end_points = [(test_labels[i], test_end_points[i]) for i in range(len(test_labels))]

    # Create the test dataset and dataloader with labels and end points (for the regression head)
    test_dataset = SleepDataset(test_sequences, test_labels_and_end_points )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    ########################## PREPARE INDIVIDUAL SUBJECTS TO TEST ON  #############################
    if selected_participants is not None:
        selected_participant_bothlabels = selected_participants_data.copy()
        selected_participant_bothlabels['Old_label'] = selected_participant_bothlabels['Label'].copy()


    ############################ INITIALISE THE SCHEDULED LEARNING  ################################
    # set the sleep onset and awake thresholds to initial values
    sleep_onset_threshold = initial_sleep_onset_threshold
    awake_threshold = initial_awake_threshold

    # Calculate the deltas for the thresholds (how much the thresholds should change each iteration of the schedule until convergence)
    distance_to_convergence_sleep_onset = convergence_point - sleep_onset_threshold
    sleep_onset_delta = distance_to_convergence_sleep_onset / total_epochs
    distance_to_convergence_awake = awake_threshold - convergence_point
    awake_delta = distance_to_convergence_awake / total_epochs

    ################################## INITIALISE MODELS  #########################################
    # Initialize the classification model
    model_classification = SleepOnsetRNNClassifier(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, l2=l2).to(device)

    # Define the optimizer
    optimizer_class = torch.optim.AdamW(model_classification.parameters(), lr=lr, weight_decay=weight_decay)

    criterion_regression = CustomMSELoss()

    # Empty lists to store training and validation losses
    train_losses_reg_all, test_losses_reg_all = [], []


    ############################# BEGIN THE SCHEDULED LEARNING  ###################################

    for epoch in range(total_epochs):


        train_losses_class, test_losses_class, train_losses_reg, test_losses_reg = [], [], [], []

        # Check if the thresholds have converged
        if abs(sleep_onset_threshold - convergence_point) < convergence_sensitivity:
            if ifplot:
                print('Sleep onset threshold has converged')
            break
        if abs(awake_threshold - convergence_point) < convergence_sensitivity:
            if ifplot:
                print('Awake threshold has converged')
            break

        # Reinitilize the optimizer
        #optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

        # save each epoch in a separate folder
        save_directory_epoch = save_directory + '/epoch_' + str(epoch)
        if not os.path.exists(save_directory_epoch):
            os.makedirs(save_directory_epoch)


        sleep_onset_threshold, awake_threshold = update_thresholds(epoch, sleep_onset_threshold, awake_threshold, awake_delta, sleep_onset_delta)

        if epoch == total_epochs - 1:
          sleep_onset_threshold = convergence_point
          awake_threshold = convergence_point


        #if ifplot:
        print('New sleep onset threshold: ', sleep_onset_threshold)
        print('New awake threshold: ', awake_threshold)
        model_parameters = {'pre_sleep_window': sleep_onset_threshold,
                            'awake_window': awake_threshold,
                            'convergence_point': convergence_point,
                            'sliding_window_size': window_size_minutes}


        # Prepare the data
        train_sequences, train_labels, train_end_points, train_sleep_stages, index, label = prepare_classification_data(mydata_train = mydata_train, sleep_onset_threshold = sleep_onset_threshold,
                                                                                                                        awake_threshold = awake_threshold ,
                                                                                                                        method = method, filling_limit = filling_limit,
                                                                                                                        window_size_minutes = window_size_minutes, ifmissing = ifmissing,
                                                                                                                        random_seed = random_seed, resampling_method = resampling_method)
        #print('Train sequences: ', train_sequences.shape)
        #print('Train labels: ', train_labels.shape)
        # adjust the classification training loss accordidng to the class proportion in the training set to prevent imbalance
        # only works if ifclassweights = 1
        criterion_class, pos_weight = define_loss_function(ifclassweights, train_labels, device)

        # Create the training set and dataloader
        train_labels_and_end_points = [(train_labels[i], train_end_points[i]) for i in range(len(train_labels))]

        train_dataset = SleepDataset(train_sequences, train_labels_and_end_points)
        if shuffle_train_loader:
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        else:
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

        print('--------------------------------------Evaluate classification ----------------------------------------')

        ########################## TRAIN AND EVALUATE  #############################
        train_losses_class, test_losses_class, model_classification, optimizer_class, test_predictions_class, test_actuals_class, test_prediction_probabilities_class, test_metrics_class = train_and_evaluate_classification_model(model = model_classification, train_loader = train_loader,
                                                                                                                                                                                                                                    label = label, test_labels = test_labels,
                                                                                                                                                                                                                                    test_end_points = test_end_points,
                                                                                                                                                                                                                                    test_sleep_stages= test_sleep_stages,
                                                                                                                                                                                                                                    test_loader = test_loader, device = device,
                                                                                                                                                                                                                                    criterion = criterion_class,
                                                                                                                                                                                                                                    optimizer = optimizer_class,
                                                                                                                                                                                                                                    num_epochs = num_epochs,
                                                                                                                                                                                                                                    index = index,
                                                                                                                                                                                                                                    ifprobabilities = ifprobabilitiesanalysis,
                                                                                                                                                                                                                                    ifplot = ifplot,
                                                                                                                                                                                                                                    ifsaveplots=ifsaveplots,
                                                                                                                                                                                                                                    save_directory_epoch = save_directory_epoch,
                                                                                                                                                                                                                                    awake_threshold= awake_threshold,
                                                                                                                                                                                                                                    sleep_onset_threshold=sleep_onset_threshold,
                                                                                                                                                                                                                                    window_size_minutes=window_size_minutes)

        # plot the losses for the classification model
        title = 'Training and validation losses during classification with window size of ' + str(window_size_minutes) + ' minutes'
        filename = 'classification_losses.png'
        plot_losses(train_losses_class, test_losses_class, title, ifplot, ifsaveplots,
                    save_directory = save_directory_epoch, filename = filename)

        # Save the pos class weight in the test metrics dictionary
        if ifclassweights:
            test_metrics_class['pos_class_weight'] = pos_weight.cpu().detach().numpy()[0]


        # Get the stats for probability change points and mean prediction times in participants on the validaiton set
        _,  participant_dict = test_on_random_participants(data = mydata_test, model =  model_classification, criterion = criterion_class, random_seed = random_seed,
                                                                                            ifplot = False, device = device, output_path_new =  save_directory_epoch, index = index,
                                                                                            convergence_point = convergence_point,
                                                                                            awake_window = awake_threshold, pre_sleep_window = sleep_onset_threshold,
                                                                                            ifsaveplots = False, window_size_minutes = window_size_minutes, ifmissing = True,
                                                                                            filling_limit = filling_limit, method = method, resampling_method = None)


        if selected_participants is not None:
            participant_dataloaders, _ = test_on_random_participants(data = selected_participant_bothlabels, model = model_classification, criterion = criterion_class,
                                                                                                random_seed = random_seed, ifplot = ifplot, device = device, output_path_new =  save_directory_epoch, index = index,
                                                                                                convergence_point = convergence_point,
                                                                                                awake_window = awake_threshold, pre_sleep_window = sleep_onset_threshold, ifsaveplots = True,
                                                                                                window_size_minutes = window_size_minutes, ifmissing = True,
                                                                                                filling_limit = filling_limit, method = method, resampling_method = None)



        # From the pretrained classification model, create a new model for regression and train it
        model_regression = PretrainedModelForRegression(model_classification).to(device)
        optimizer_reg = torch.optim.AdamW(model_regression.parameters(), lr=lr, weight_decay=weight_decay)

        print('--------------------------------------Evaluate regression ----------------------------------------')

        train_losses_reg, test_losses_reg, model_regression, optimizer_reg, test_predictions_red, test_actuals, test_metrics_reg = train_and_evaluate_regression_model(model = model_regression, train_loader =train_loader,
                                                                                                                                        test_loader = test_loader, criterion = criterion_regression,
                                                                                                                                        optimizer = optimizer_reg, device = device, num_epochs = num_epochs, ifplot = ifplot,
                                                                                                                                        ifsaveplots = ifsaveplots, output_path = save_directory_epoch)
        if selected_participants is not None:
            test_reg_on_random_participants(participant_dataloaders = participant_dataloaders, model = model_regression,  criterion = criterion_regression,
                                                 device = device, random_seed = random_seed, ifplot = ifplot, output_path_new =save_directory_epoch, index = index)



        # After each epoch, save the model
        save_models(model_classification, model_regression, save_directory_epoch, sleep_onset_threshold, awake_threshold, window_size_minutes)

        # plot the losses for the regression model
        filename = 'regression_losses.png'
        title = 'Training and validation losses during regression with window size of ' + str(window_size_minutes) + ' minutes'
        plot_losses(train_losses_reg, test_losses_reg, title,
                    ifplot, ifsaveplots,
                    save_directory = save_directory_epoch, filename = filename)

        # Add results to dataframe
        ifconverged = 0
        if epoch + 1 == total_epochs:
            ifconverged = 1

        model_parameters['ifconverged'] = ifconverged
        model_parameters['initial_sleep_onset_threshold'] = initial_sleep_onset_threshold

        # joing two dictionaries
        test_metrics = {**model_parameters, **test_metrics_class,  **participant_dict, **test_metrics_reg}
        # populate the results_df dataframe with the results from the current epoch
        new_row = pd.DataFrame(test_metrics, index = [epoch])

        # Open the results dataframe and add the new results
        results_df_all = pd.read_csv(os.path.join(results_saving_dir, 'results.csv'))
        results_df_all = pd.concat([results_df_all, new_row], ignore_index=True)
        results_df_all.to_csv(os.path.join(results_saving_dir, 'results.csv'), index=False)
        del results_df_all
        del new_row
        del test_metrics
        del train_loader

    # flatten the list of lists
    test_losses_reg_all = [item for sublist in test_losses_reg_all for item in sublist]
    train_losses_reg_all = [item for sublist in train_losses_reg_all for item in sublist]

    # plot the losses across all epochs
    filename = 'all_regression_losses.png'
    title = 'Training and validation losses during all regression with window size of ' + str(window_size_minutes) + ' minutes'
    plot_losses(train_losses_reg_all, test_losses_reg_all, title,
                ifplot, ifsaveplots,
                save_directory = save_directory, filename = filename)


    # Clean up the cuda memory
    torch.cuda.empty_cache()
    # close all the plots
    plt.close('all')
    # clean up the memory
    gc.collect()

    del mydata_train, mydata_test





In [42]:
selected_participants = np.array(selected_participants)

In [43]:
selected_participants

array([ 110,  188,  912,  933, 1077, 2019, 2251, 2443, 2574, 2780, 3689,
       4512, 4618, 4998, 5255, 5568, 5801, 6279, 6321, 6475])

In [44]:
selected_participants_data

,Label,Sbj_ID,Age,Gender,Race1,Race2,Race3,Race4,ifCleanOnset,Time2Sleep,...,wrseltr5,slpapnea5,cpap5,dntaldv5,uvula5,insmnia5,rstlesslgs5,whiirs5c,epslpscl5c,hoostmeq5c
0,29.9,110,60,1,1.0,0.0,0.0,0.0,1,93.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,11.0
1,29.8,110,60,1,1.0,0.0,0.0,0.0,1,93.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,11.0
2,29.7,110,60,1,1.0,0.0,0.0,0.0,1,93.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,11.0
3,29.6,110,60,1,1.0,0.0,0.0,0.0,1,93.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,11.0
4,29.5,110,60,1,1.0,0.0,0.0,0.0,1,93.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.4,6475,63,1,0.0,1.0,0.0,0.0,1,77.5,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,16.0
5996,0.3,6475,63,1,0.0,1.0,0.0,0.0,1,77.5,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,16.0
5997,0.2,6475,63,1,0.0,1.0,0.0,0.0,1,77.5,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,16.0
5998,0.1,6475,63,1,0.0,1.0,0.0,0.0,1,77.5,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,16.0


In [45]:
convergence_points =[4]

In [46]:
convergence_points = [4, 6, 8, 10, 12, 14]
convergence_starting_points4 = [2]
convergence_starting_points6 = [2, 4]
convergence_starting_points8 = [2, 4, 6]
convergence_starting_points10_14 = [2, 4, 6, 8]
convergence_starting_points = {4: convergence_starting_points4, 6: convergence_starting_points6, 8: convergence_starting_points8, 10: convergence_starting_points10_14, 12: convergence_starting_points10_14, 14: convergence_starting_points10_14}

window_size2 = [0.5, 1, 1.5]
window_size4 = [0.5, 1, 2, 3]
window_size6_8= [0.5, 1, 2, 3, 4, 5]
starting_window_size = {4: window_size4, 6: window_size6_8, 8: window_size6_8}

In [47]:
def hyperparameter_tuning_loop(mydata, convergence_starting_points_dict, starting_window_size_dict,
                            resampling_method = 'undersampling', results_saving_dir = 'regression_head_rnn_schedule_learning',
                            selected_participants = None, selected_participants_data = None, ifresetresults = False):

    if not os.path.exists(results_saving_dir):
        os.makedirs(results_saving_dir)

    # Initialize a DataFrame to store results
        # Initialize a DataFrame to store results
    columns = ['convergence_point', 'initial_sleep_onset_threshold','ifconverged', 'pre_sleep_window', 'awake_window', 'pos_class_weight',
                'sliding_window_size', 'Accuracy', 'precision', 'recall', 'f1_weighted',
                'f1_macro', 'f1_pre_sleep', 'f1_awake', 'auc',
                'one_crossing_percentage', 'mean_time_of_prediction',
                'std_time_of_prediction', 'one_crossing_percentage_smoothed',
                'mean_time_of_prediction_smoothed', 'std_time_of_prediction_smoothed',
                'MAE', 'MSE', 'RMSE', 'R2', 'custom_MSE']

    if ifresetresults:
        results_df_all = pd.DataFrame(columns=columns)
        results_df_all.to_csv(os.path.join(results_saving_dir, 'results.csv'))


    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device('cpu')
    print("Using {}.".format(device))
    for convergence_point in list(convergence_starting_points_dict.keys()):
        for starting_point in convergence_starting_points_dict[convergence_point]:
            window_sizes = starting_window_size_dict[starting_point]
            for window_size in window_sizes:

                gc.collect()
                torch.cuda.empty_cache()

                experiment_dir = os.path.join(results_saving_dir, f'convergence_point_{convergence_point}_starting_point_{starting_point}_window_size_{window_size}')
                if not os.path.exists(experiment_dir):
                    os.makedirs(experiment_dir)

                print('Convergence point: ', convergence_point)
                print('Starting point: ', starting_point)
                print('Window size: ', window_size)
                training_loop_RNN_regclass_schedule(mydata, convergence_point, total_epochs = 3, filling_limit = 40, method = 'LOCF', ifclassweights = 1, input_size = 86,
                                            num_epochs = 10, hidden_size = 128, num_layers = 2, dropout = 0, l2 = 0, lr = 0.001, weight_decay = 0.001, batch_size = 64,
                                            device = device, random_seed = 42, window_size_minutes = window_size, ifprobabilitiesanalysis = 1,
                                            if_stratified_sampling = 1, initial_awake_threshold = 30, initial_sleep_onset_threshold = starting_point,
                                            convergence_sensitivity = 0.05, resampling_method = resampling_method, ifplot = False, ifsaveplots = True,
                                            output_path = experiment_dir, shuffle_train_loader = True, results_saving_dir = results_saving_dir, selected_participants_data = selected_participants_data,
                                            selected_participants = selected_participants, ifresetresults = False)



In [48]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [49]:
torch.cuda.is_available()

True

## Try and see if the loop works

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
convergence_starting_points_test = {4:[2]}
starting_window_size_test = {2:[0.5, 1]}
folder_name = '/content/drive/MyDrive/sleep onset datasets/regression_head_rnn_schedule_learning_test'
hyperparameter_tuning_loop(data_train, convergence_starting_points_test, starting_window_size_test, resampling_method = 'undersampling', results_saving_dir = folder_name , selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = True)


Using cuda.
Convergence point:  4
Starting point:  2
Window size:  0.5
New sleep onset threshold:  2.7
New awake threshold:  21.3
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 58.58585858585859%
Mean time of prediction for participants with only one crossing: 11.408620689655171 minutes
STD of time of prediction for participants with only one crossing: 8.345993969497119 minutes
Percentage of participants with only one crossing (smoothed): 57.57575757575758%
Mean time of prediction for participants with only one crossing (smoothed): 10.82280701754386 minutes
STD of time of prediction for participants with only one crossing (smoothed): 8.243168730357503 minutes
Percentage of participants with only one crossing: 50.0%
Mean time of prediction for participants with only one crossing: 12.69 minutes
STD of time of prediction for participants with only one crossing: 6.817690224702205 minu

KeyboardInterrupt: ignored

## Hypereparameter tune for window selection

In [ ]:
convergence_points = [4, 6, 8, 10, 12, 14]
convergence_starting_points4 = [2]
convergence_starting_points6 = [2, 4]
convergence_starting_points8 = [2, 4, 6]
convergence_starting_points10_14 = [2, 4, 6, 8]
convergence_starting_points = {4: convergence_starting_points4, 6: convergence_starting_points6, 8: convergence_starting_points8, 10: convergence_starting_points10_14, 12: convergence_starting_points10_14, 14: convergence_starting_points10_14}

window_size2 = [0.5, 1, 1.5]
window_size4 = [0.5, 1, 2, 3]
window_size6_8= [0.5, 1, 2, 3, 4, 5]
starting_window_size = {2:window_size2, 4: window_size4, 6: window_size6_8, 8: window_size6_8}

In [ ]:
folder_name = '/content/drive/MyDrive/sleep onset datasets/FIXED_regression_head_rnn_schedule_learning_mixed_onset'

In [ ]:

convergence_starting_points = {4: [2]}
window_size2 = [0.5,1, 1.5]
starting_window_size = {2:window_size2}
hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = True)


Using cuda.
Convergence point:  4
Starting point:  2
Window size:  0.5
New sleep onset threshold:  2.7
New awake threshold:  21.3
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 74.74747474747475%
Mean time of prediction for participants with only one crossing: 11.347297297297297 minutes
STD of time of prediction for participants with only one crossing: 7.576175726667383 minutes
Percentage of participants with only one crossing (smoothed): 73.73737373737373%
Mean time of prediction for participants with only one crossing (smoothed): 10.549315068493152 minutes
STD of time of prediction for participants with only one crossing (smoothed): 7.360429619825399 minutes
Percentage of participants with only one crossing: 55.00000000000001%
Mean time of prediction for participants with only one crossing: 13.718181818181819 minutes
STD of time of prediction for participants with only one cross

In [ ]:

convergence_starting_points = {6: [2]}
window_size2 = [0.5, 1, 1.5]
starting_window_size = {2:window_size2}
hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  6
Starting point:  2
Window size:  0.5
New sleep onset threshold:  3.3
New awake threshold:  22.0
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 63.63636363636363%
Mean time of prediction for participants with only one crossing: 12.352380952380953 minutes
STD of time of prediction for participants with only one crossing: 7.388727088061882 minutes
Percentage of participants with only one crossing (smoothed): 70.70707070707071%
Mean time of prediction for participants with only one crossing (smoothed): 11.042857142857146 minutes
STD of time of prediction for participants with only one crossing (smoothed): 7.724054661132535 minutes
Percentage of participants with only one crossing: 40.0%
Mean time of prediction for participants with only one crossing: 12.45 minutes
STD of time of prediction for participants with only one crossing: 7.846018098373212 min

In [ ]:

convergence_starting_points = {6: [4]}
window_size4 = [0.5, 1, 2, 3]
starting_window_size = {4:window_size4}
hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  6
Starting point:  4
Window size:  0.5
New sleep onset threshold:  4.7
New awake threshold:  22.0
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 60.60606060606061%
Mean time of prediction for participants with only one crossing: 12.141666666666664 minutes
STD of time of prediction for participants with only one crossing: 6.627978366155266 minutes
Percentage of participants with only one crossing (smoothed): 64.64646464646465%
Mean time of prediction for participants with only one crossing (smoothed): 10.098437500000001 minutes
STD of time of prediction for participants with only one crossing (smoothed): 6.602070986334042 minutes
Percentage of participants with only one crossing: 45.0%
Mean time of prediction for participants with only one crossing: 15.122222222222222 minutes
STD of time of prediction for participants with only one crossing: 7.337086

In [ ]:

convergence_starting_points = {6: [4]}
window_size4 = [2, 3]
starting_window_size = {4:window_size4}
hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  6
Starting point:  4
Window size:  2
New sleep onset threshold:  4.7
New awake threshold:  22.0
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 41.41414141414141%
Mean time of prediction for participants with only one crossing: 19.704878048780486 minutes
STD of time of prediction for participants with only one crossing: 8.809389425964698 minutes
Percentage of participants with only one crossing (smoothed): 46.464646464646464%
Mean time of prediction for participants with only one crossing (smoothed): 16.428260869565225 minutes
STD of time of prediction for participants with only one crossing (smoothed): 9.04079313189314 minutes
Percentage of participants with only one crossing: 45.0%
Mean time of prediction for participants with only one crossing: 21.66666666666667 minutes
STD of time of prediction for participants with only one crossing: 8.606586624

In [ ]:
convergence_starting_points = {8: [2]}
window_size2 = [1, 1.5]
starting_window_size = {2:window_size2}
older_name = '/content/drive/MyDrive/sleep onset datasets/regression_head_rnn_schedule_learning_mixed_onset'
hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  8
Starting point:  2
Window size:  0.5
New sleep onset threshold:  4.0
New awake threshold:  22.7
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 70.70707070707071%
Mean time of prediction for participants with only one crossing: 11.467142857142857 minutes
STD of time of prediction for participants with only one crossing: 6.905768218951281 minutes
Percentage of participants with only one crossing (smoothed): 66.66666666666666%
Mean time of prediction for participants with only one crossing (smoothed): 11.268181818181821 minutes
STD of time of prediction for participants with only one crossing (smoothed): 6.966307640978851 minutes
Percentage of participants with only one crossing: 60.0%
Mean time of prediction for participants with only one crossing: 11.4 minutes
STD of time of prediction for participants with only one crossing: 7.98373346248483 minut

In [ ]:
folder_name

'/content/drive/MyDrive/sleep onset datasets/FIXED_regression_head_rnn_schedule_learning_mixed_onset'

In [ ]:

convergence_starting_points = {8: [4]}
window_size4 = [0.5, 1, 2, 3]
starting_window_size = {4:window_size4}
hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  8
Starting point:  4
Window size:  0.5
New sleep onset threshold:  5.3
New awake threshold:  22.7
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 71.71717171717171%
Mean time of prediction for participants with only one crossing: 12.590140845070422 minutes
STD of time of prediction for participants with only one crossing: 7.483712924417413 minutes
Percentage of participants with only one crossing (smoothed): 66.66666666666666%
Mean time of prediction for participants with only one crossing (smoothed): 12.374242424242425 minutes
STD of time of prediction for participants with only one crossing (smoothed): 7.658176668151789 minutes
Percentage of participants with only one crossing: 60.0%
Mean time of prediction for participants with only one crossing: 14.308333333333332 minutes
STD of time of prediction for participants with only one crossing: 9.139881

In [ ]:

convergence_starting_points = {8: [4]}
window_size4 = [3]
starting_window_size = {4:window_size4}
hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  8
Starting point:  4
Window size:  3
New sleep onset threshold:  5.3
New awake threshold:  22.7
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 52.52525252525253%
Mean time of prediction for participants with only one crossing: 14.44230769230769 minutes
STD of time of prediction for participants with only one crossing: 8.256683683161599 minutes
Percentage of participants with only one crossing (smoothed): 51.515151515151516%
Mean time of prediction for participants with only one crossing (smoothed): 13.631372549019611 minutes
STD of time of prediction for participants with only one crossing (smoothed): 8.314893092260238 minutes
Percentage of participants with only one crossing: 50.0%
Mean time of prediction for participants with only one crossing: 14.690000000000001 minutes
STD of time of prediction for participants with only one crossing: 8.65383729

In [ ]:

convergence_starting_points = {8: [6]}
window_size6_8= [0.5, 1, 2, 3, 4, 5]
starting_window_size = {6:window_size6_8}
hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  8
Starting point:  6
Window size:  0.5


KeyboardInterrupt: ignored

In [ ]:

convergence_starting_points = {8: [6]}
window_size6_8= [3, 4, 5]
starting_window_size = {6:window_size6_8}

hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  8
Starting point:  6
Window size:  3
New sleep onset threshold:  6.7
New awake threshold:  22.7
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 46.464646464646464%
Mean time of prediction for participants with only one crossing: 15.319565217391306 minutes
STD of time of prediction for participants with only one crossing: 8.875922966464742 minutes
Percentage of participants with only one crossing (smoothed): 45.45454545454545%
Mean time of prediction for participants with only one crossing (smoothed): 15.377777777777784 minutes
STD of time of prediction for participants with only one crossing (smoothed): 8.997009105225256 minutes
Percentage of participants with only one crossing: 40.0%
Mean time of prediction for participants with only one crossing: 14.2625 minutes
STD of time of prediction for participants with only one crossing: 7.25257497375932 min

In [ ]:

convergence_starting_points = {12: [2]}
window_size2 = [0.5, 1, 1.5]
starting_window_size = {2: window_size4}

hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  12
Starting point:  2
Window size:  0.5
New sleep onset threshold:  5.3
New awake threshold:  24.0
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 64.64646464646465%
Mean time of prediction for participants with only one crossing: 13.628125 minutes
STD of time of prediction for participants with only one crossing: 6.930107970614528 minutes
Percentage of participants with only one crossing (smoothed): 64.64646464646465%
Mean time of prediction for participants with only one crossing (smoothed): 13.121875 minutes
STD of time of prediction for participants with only one crossing (smoothed): 7.701571851536217 minutes
Percentage of participants with only one crossing: 45.0%
Mean time of prediction for participants with only one crossing: 17.877777777777776 minutes
STD of time of prediction for participants with only one crossing: 9.298599390084721 minutes

In [ ]:

convergence_starting_points = {12: [4]}
window_size4 = [0.5, 1, 2, 3]
starting_window_size = {4: window_size4}

hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  12
Starting point:  4
Window size:  0.5
New sleep onset threshold:  6.7
New awake threshold:  24.0
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 68.68686868686868%
Mean time of prediction for participants with only one crossing: 13.286764705882353 minutes
STD of time of prediction for participants with only one crossing: 7.4055206833615355 minutes
Percentage of participants with only one crossing (smoothed): 61.61616161616161%
Mean time of prediction for participants with only one crossing (smoothed): 13.167213114754103 minutes
STD of time of prediction for participants with only one crossing (smoothed): 7.273277415609704 minutes
Percentage of participants with only one crossing: 45.0%
Mean time of prediction for participants with only one crossing: 15.166666666666666 minutes
STD of time of prediction for participants with only one crossing: 8.3313

In [ ]:

convergence_starting_points = {12: [4]}
window_size4 = [3]
starting_window_size = {4: window_size4}

hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  12
Starting point:  4
Window size:  3
New sleep onset threshold:  6.7
New awake threshold:  24.0
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 55.55555555555556%
Mean time of prediction for participants with only one crossing: 15.36181818181818 minutes
STD of time of prediction for participants with only one crossing: 6.098181276088229 minutes
Percentage of participants with only one crossing (smoothed): 55.55555555555556%
Mean time of prediction for participants with only one crossing (smoothed): 15.009090909090915 minutes
STD of time of prediction for participants with only one crossing (smoothed): 6.203731215303863 minutes
Percentage of participants with only one crossing: 35.0%
Mean time of prediction for participants with only one crossing: 15.814285714285715 minutes
STD of time of prediction for participants with only one crossing: 5.34427024

In [ ]:

convergence_starting_points = {12: [6]}
window_size6_8= [0.5, 1, 2, 3, 4, 5]
starting_window_size = {6:window_size6_8}

hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  12
Starting point:  6
Window size:  0.5
New sleep onset threshold:  8.0
New awake threshold:  24.0
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 66.66666666666666%
Mean time of prediction for participants with only one crossing: 14.434848484848487 minutes
STD of time of prediction for participants with only one crossing: 7.21354980121569 minutes
Percentage of participants with only one crossing (smoothed): 58.58585858585859%
Mean time of prediction for participants with only one crossing (smoothed): 13.65689655172414 minutes
STD of time of prediction for participants with only one crossing (smoothed): 6.926513579616284 minutes
Percentage of participants with only one crossing: 50.0%
Mean time of prediction for participants with only one crossing: 13.580000000000002 minutes
STD of time of prediction for participants with only one crossing: 8.0674407

In [ ]:

convergence_starting_points = {12: [6]}
window_size6_8= [2, 3, 4, 5]
starting_window_size = {6:window_size6_8}

hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  12
Starting point:  6
Window size:  2
New sleep onset threshold:  8.0
New awake threshold:  24.0
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 65.65656565656566%
Mean time of prediction for participants with only one crossing: 14.738461538461538 minutes
STD of time of prediction for participants with only one crossing: 6.318000957159991 minutes
Percentage of participants with only one crossing (smoothed): 62.62626262626263%
Mean time of prediction for participants with only one crossing (smoothed): 14.224193548387102 minutes
STD of time of prediction for participants with only one crossing (smoothed): 6.999670164668613 minutes
Percentage of participants with only one crossing: 30.0%
Mean time of prediction for participants with only one crossing: 14.166666666666666 minutes
STD of time of prediction for participants with only one crossing: 4.6460257

In [ ]:

convergence_starting_points = {12: [6]}
window_size6_8= [5]
starting_window_size = {6:window_size6_8}

hyperparameter_tuning_loop(data_train, convergence_starting_points, starting_window_size , resampling_method = 'undersampling', results_saving_dir = folder_name, selected_participants = selected_participants, selected_participants_data = selected_participants_data, ifresetresults = False)


Using cuda.
Convergence point:  12
Starting point:  6
Window size:  5
New sleep onset threshold:  8.0
New awake threshold:  24.0
--------------------------------------Evaluate classification ----------------------------------------
Percentage of participants with only one crossing: 60.60606060606061%
Mean time of prediction for participants with only one crossing: 15.303333333333331 minutes
STD of time of prediction for participants with only one crossing: 5.90939835253039 minutes
Percentage of participants with only one crossing (smoothed): 61.61616161616161%
Mean time of prediction for participants with only one crossing (smoothed): 14.536065573770497 minutes
STD of time of prediction for participants with only one crossing (smoothed): 5.850403735175252 minutes
Percentage of participants with only one crossing: 40.0%
Mean time of prediction for participants with only one crossing: 15.1375 minutes
STD of time of prediction for participants with only one crossing: 6.4909432095805615 mi

## Analysing Hyperparameter Tuning results

In [ ]:
output_path = folder_name
results_df_all = pd.read_csv(f'{output_path}/results.csv')
results_df_all = results_df_all.drop(columns=['Unnamed: 0'])
# only keep results where ifconverged = 1
results_df_all_selected = results_df_all[results_df_all['ifconverged'] == 1]
#results_df_all_selected = results_df_all[results_df_all['ifconverged'] == 1]
# show how many results we have



#Only keep the colums:
#Only keep the colums:
columns = ['convergence_point', 'initial_sleep_onset_threshold', 'sliding_window_size', 'f1_weighted', 'f1_pre_sleep',
       'f1_awake', 'precision',
       'recall', 'auc', 'one_crossing_percentage',
       'mean_time_of_prediction',
       'std_time_of_prediction']
results_df_all_selected = results_df_all_selected[columns]



sorted_results = results_df_all_selected.sort_values(by=['f1_weighted'], ascending=False)
# make the values in the table only have 2 decimal places after saving
sorted_results = sorted_results.round(2)

headers = ['t_{PS}', 't_0_{PS}', 't_{SW}', 'F1_W', 'F1_{PS}','F1_{AW}', 'P', 'S', 'AUC', 'R_ps', 'Mean(t_{pred})', 'STD(t_{pred})']
sorted_results.columns = headers
# Save only the top 3 results
sorted_results = sorted_results.head(5)

sorted_results.to_latex(f'{output_path}/results_top5_f1.tex', index=False, float_format="%.2f")

print(sorted_results.head(5))

    t_{PS}  t_0_{PS}  t_{SW}  F1_W  F1_{PS}  F1_{AW}     P     S   AUC   R_ps  \
8        4         2     1.5  0.85     0.90     0.54  0.46  0.66  0.84  13.13   
2        4         2     0.5  0.83     0.89     0.50  0.40  0.66  0.83  39.39   
5        4         2     1.0  0.83     0.88     0.51  0.40  0.69  0.83  42.42   
30       6         4     3.0  0.81     0.87     0.62  0.56  0.69  0.84  28.28   
17       6         2     1.5  0.79     0.85     0.57  0.49  0.68  0.81  40.40   

    Mean(t_{pred})  STD(t_{pred})  
8            11.42           4.80  
2             7.44           5.95  
5             6.77           6.43  
30           13.28           8.81  
17            9.30           5.62  


<ipython-input-47-7635dfc7118c>:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  sorted_results.to_latex(f'{output_path}/results_top5_f1.tex', index=False, float_format="%.2f")


In [ ]:
output_path = folder_name
results_df_all = pd.read_csv(f'{output_path}/results.csv')
results_df_all = results_df_all.drop(columns=['Unnamed: 0'])
# only keep results where ifconverged = 1
results_df_all_selected = results_df_all[results_df_all['ifconverged'] == 1]
#results_df_all_selected = results_df_all[results_df_all['ifconverged'] == 1]
# show how many results we have


#Only keep the colums:
#Only keep the colums:
columns = ['convergence_point', 'initial_sleep_onset_threshold', 'sliding_window_size','one_crossing_percentage',
       'mean_time_of_prediction',
       'std_time_of_prediction', 'f1_weighted', 'f1_pre_sleep',
       'f1_awake', 'precision',
       'recall', 'auc']
results_df_all_selected = results_df_all_selected[columns]



sorted_results = results_df_all_selected.sort_values(by=['one_crossing_percentage'], ascending=False)
# make the values in the table only have 2 decimal places after saving
sorted_results = sorted_results.round(2)

headers = ['t_{PS}', 't_0_{PS}', 't_{SW}', 'R_ps', 'Mean(t_{pred})', 'STD(t_{pred})', 'F1_W', 'F1_{PS}','F1_{AW}', 'P', 'S', 'AUC']
sorted_results.columns = headers
# Save only the top 3 results
sorted_results = sorted_results.head(5)

sorted_results.to_latex(f'{output_path}/results_top5_crossing.tex', index=False, float_format="%.2f")

print(sorted_results.head(5))

     t_{PS}  t_0_{PS}  t_{SW}   R_ps  Mean(t_{pred})  STD(t_{pred})  F1_W  \
94       12         6     1.0  72.73           13.78           6.97  0.66   
36        8         2     1.0  71.72           10.39           6.96  0.74   
102      12         6     3.0  71.72           13.47           5.12  0.69   
53        8         6     1.0  66.67           12.71           8.10  0.70   
99       12         6     2.0  65.66           12.97           6.13  0.69   

     F1_{PS}  F1_{AW}     P     S   AUC  
94      0.68     0.63  0.57  0.70  0.72  
36      0.80     0.58  0.51  0.69  0.79  
102     0.69     0.69  0.63  0.75  0.76  
53      0.75     0.56  0.46  0.73  0.77  
99      0.73     0.64  0.64  0.65  0.73  


<ipython-input-50-5c2e53519f2a>:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  sorted_results.to_latex(f'{output_path}/results_top5_crossing.tex', index=False, float_format="%.2f")


In [ ]:
convergence_points = [4,12]
initial_sleep_onset_thresholds = [2, 6]
sliding_window_sizes = [1.5, 3]

## Test the best models on the test set

### Helper functions

In [50]:
def all_test_classification_model(model, label,  test_labels, test_end_points,
                                test_sleep_stages,
                                test_loader, device, criterion,
                                index, ifprobabilities,
                                ifplot, ifsaveplots, save_directory_epoch,
                                awake_threshold, sleep_onset_threshold, window_size_minutes):
    if ifprobabilities:
        test_predictions, test_actuals, test_prediction_probabilities = test_classification(model, test_loader, device, criterion, index, threshold=0.5, ifprobabilities=ifprobabilities, ifplot = ifplot)

        test_metrics = evaluate_classification(true_labels=test_actuals, predictions=test_predictions,
                                               prediction_probabilities=test_prediction_probabilities,
                                               index=index, columns = label, ifsaveplots=ifsaveplots,
                                               output_path=save_directory_epoch, ifplot=ifplot)

        if ifplot:
                print('------------------------------------------- No averaging ----------------------------------------')
                print('Accuracy')
        accuracies, precision, recall, f1 = check_accuracy_timeline(test_predictions, test_labels, test_end_points,
                                                                    test_sleep_stages, window_for_averaging= 1, awake_window=awake_threshold,
                                                                    pre_sleep_window=sleep_onset_threshold, rnn_window = window_size_minutes,
                                                                    ifsaveplots = ifsaveplots, output_path= save_directory_epoch, ifplot = ifplot)
        if ifplot:
            print('Probability of sleep as predicted by the model')
        y_pred_sorted = check_accuracy_timeline(test_prediction_probabilities, test_labels, test_end_points,
                                                test_sleep_stages, window_for_averaging= 1, if_proba = True,
                                                classes = index, awake_window=awake_threshold, pre_sleep_window=sleep_onset_threshold,
                                                rnn_window = window_size_minutes, ifsaveplots = ifsaveplots, output_path= save_directory_epoch,
                                                ifplot = ifplot)
        if ifplot:
            print('-------------------------------Averaging of accuracy over 30 seconds---------------------------------')
            print('Accuracy')
        accuracies, precision, recall, f1 = check_accuracy_timeline(test_predictions, test_labels, test_end_points,
                                                                    test_sleep_stages, window_for_averaging= 5,
                                                                    awake_window=awake_threshold, pre_sleep_window=sleep_onset_threshold,
                                                                    rnn_window = window_size_minutes, ifsaveplots = ifsaveplots,
                                                                    output_path= save_directory_epoch, ifplot = ifplot)
        if ifplot:
            print('Probability of sleep as predicted by the model')
        y_pred_sorted = check_accuracy_timeline(test_prediction_probabilities, test_labels, test_end_points,
                                                test_sleep_stages, window_for_averaging= 5, if_proba = True,
                                                classes = index,  awake_window=awake_threshold,
                                                pre_sleep_window=sleep_onset_threshold, rnn_window = window_size_minutes,
                                                ifsaveplots = ifsaveplots, output_path= save_directory_epoch, ifplot = ifplot)

        return  test_predictions, test_actuals, test_prediction_probabilities, test_metrics

    else:
        test_predictions, test_actuals = test_classification(model, test_loader, device, criterion, index, threshold=0.5, ifprobabilities=ifprobabilities, ifplot = ifplot)


        test_metrics = evaluate_classification(true_labels=test_actuals, predictions=test_predictions,
                                               prediction_probabilities=None,
                                               index=index, columns = label, ifsaveplots=ifsaveplots,
                                               output_path=save_directory_epoch, ifplot=ifplot)

        return  test_predictions, test_actuals, None, test_metrics


In [51]:
from sklearn.utils.class_weight import compute_class_weight
import warnings
def test_loop_RNN_regclass_schedule(test_data, models_to_test, convergence_points, initial_sleep_onset_thresholds, sliding_window_sizes,
                                    filling_limit, method, batch_size, results_saving_dir = None, ifresetresults = True, random_seed = 42):

    ################## INITIALISATION OF RESULTS STORAGE ##################
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)

    #if results_saving_dir is None:
     #   results_saving_dir = output_path

    save_directory = results_saving_dir

    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # Initialize a DataFrame to store results
    columns = ['convergence_point', 'initial_sleep_onset_threshold',
                'sliding_window_size', 'accuracy', 'precision', 'recall', 'f1_weighted',
                'f1_macro', 'f1_pre_sleep', 'f1_awake', 'auc',
                'one_crossing_percentage', 'mean_time_of_prediction',
                'std_time_of_prediction', 'one_crossing_percentage_smoothed',
                'mean_time_of_prediction_smoothed', 'std_time_of_prediction_smoothed', 'Accuracy'
                'MAE', 'MSE', 'RMSE', 'R2', 'custom_MSE']

    if ifresetresults:
        results_df = pd.DataFrame(columns=columns)
        results_df.to_csv(os.path.join(results_saving_dir, 'results_final.csv'))

    for model, convergence_point, initial_sleep_onset_threshold, sliding_window_size in zip(models_to_test, convergence_points, initial_sleep_onset_thresholds, sliding_window_sizes):

      ################## PREPARE DATA FOR TRAINING AND TESTING ##################
      # Firstly, get the datasets for a classification and regressions problem
      # Create a new column 'old_label' to store the original label
      test_data_bothlabels = test_data.copy()

      test_data_bothlabels['Old_label'] = test_data_bothlabels['Label'].copy()

      test_sequences, test_labels, test_end_points, test_sleep_stages, index, label = prepare_classification_data(mydata_train = test_data_bothlabels, sleep_onset_threshold = convergence_point,
                                                                                                                          awake_threshold = convergence_point,
                                                                                                                   method = method, filling_limit = filling_limit,
                                                                                                                  window_size_minutes = sliding_window_size,
                                                                                                                  ifmissing = True,
                                                                                                                  random_seed = random_seed, resampling_method = None)

      test_labels_and_end_points = [(test_labels[i], test_end_points[i]) for i in range(len(test_labels))]

      # Create the test dataset and dataloader with labels and end points (for the regression head)
      test_dataset = SleepDataset(test_sequences, test_labels_and_end_points )
      test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

      model_parameters = {'convergence_point': convergence_point,
                          'initial_sleep_onset_threshold': initial_sleep_onset_threshold,
                          'sliding_window_size': sliding_window_size}

      criterion= nn.BCEWithLogitsLoss()

      model_string = f'{convergence_point}_convpoint_{initial_sleep_onset_threshold}_init_{sliding_window_size}_window'
      save_directory_model =  save_directory + '/' + model_string
      if not os.path.exists(save_directory_model):
          os.makedirs(save_directory_model)


      ########################## TRAIN AND EVALUATE  #############################
      test_predictions_class, test_actuals_class, test_prediction_probabilities_class, test_metrics_class = all_test_classification_model(model = model,
                                                                                                                                          label = label, test_labels = test_labels,
                                                                                                                                          test_end_points = test_end_points,
                                                                                                                                          test_sleep_stages= test_sleep_stages,
                                                                                                                                          test_loader = test_loader, device = device,
                                                                                                                                          criterion = criterion, index = index,
                                                                                                                                          ifprobabilities = True,
                                                                                                                                          ifplot = False,
                                                                                                                                          ifsaveplots=True,
                                                                                                                                          save_directory_epoch = save_directory_model,
                                                                                                                                          awake_threshold= convergence_point,
                                                                                                                                          sleep_onset_threshold=convergence_point,
                                                                                                                                          window_size_minutes= sliding_window_size)
      # Get the stats for probability change points and mean prediction times in participants on the validaiton set
      participant_dataloaders,  participant_dict = test_on_random_participants(data = test_data_bothlabels, model =  model, criterion = criterion, random_seed = random_seed,
                                                                                          ifplot = False, device = device, output_path_new =  save_directory_model, index = index,
                                                                                          convergence_point = convergence_point,
                                                                                          awake_window = convergence_point, pre_sleep_window = convergence_point,
                                                                                          ifsaveplots = True, window_size_minutes = sliding_window_size, ifmissing = True,
                                                                                          filling_limit = filling_limit, method = method, resampling_method = None)


      # joing two dictionaries
      test_metrics = {**model_parameters, **test_metrics_class,  **participant_dict}
      # populate the results_df dataframe with the results from the current epoch
      new_row = pd.DataFrame(test_metrics, index = [0])

      # Open the results dataframe and add the new results
      results_df_all = pd.read_csv(os.path.join(results_saving_dir, 'results_final.csv'))
      results_df_all = pd.concat([results_df_all, new_row], ignore_index=True)
      results_df_all.to_csv(os.path.join(results_saving_dir, 'results_final.csv'), index=False)
      del results_df_all
      del new_row
      del test_metrics

      # Clean up the cuda memory
      torch.cuda.empty_cache()
      # close all the plots
      plt.close('all')
      # clean up the memory
      gc.collect()



In [52]:
def extract_models_from_folder(folder_name,  convergence_points, initial_sleep_onset_thresholds, sliding_window_sizes, epoch = 2):
  models = []
  for convergence_point, initial_sleep_onset_threshold, sliding_window in zip(convergence_points, initial_sleep_onset_thresholds, sliding_window_sizes):
      model_dir = folder_name + '/' + f'convergence_point_{convergence_point}_starting_point_{initial_sleep_onset_threshold}_window_size_{sliding_window}'
      epoch_dir = model_dir + '/epoch_' + str(epoch)
      model_name = f'model_classification_presleep_{convergence_point}_awake_{convergence_point}_window_{sliding_window}.pt'
      model_dict = torch.load(epoch_dir + '/' + model_name)
      model = SleepOnsetRNNClassifier(input_size=86, hidden_size=128,
                                                     num_layers=2, dropout=0.0, l2=0.0).to(device)
      model.load_state_dict(model_dict)

      models.append(model)

  return models


### Actually test it

In [53]:
folder_name = '/content/drive/MyDrive/sleep onset datasets/FIXED_regression_head_rnn_schedule_learning_mixed_onset'
convergence_points = [4,12]
initial_sleep_onset_thresholds = [2, 6]
sliding_window_sizes = [1.5, 1]

In [54]:
models_to_test = extract_models_from_folder(folder_name,  convergence_points, initial_sleep_onset_thresholds, sliding_window_sizes)

In [55]:
results_saving_dir = folder_name + '/test_best_models'
test_loop_RNN_regclass_schedule(data_test, models_to_test, convergence_points, initial_sleep_onset_thresholds, sliding_window_sizes,
                                filling_limit = 40, method = 'LOCF', batch_size = 64, results_saving_dir = results_saving_dir, ifresetresults = True, random_seed = 42)

Participant 1 of 55
Participant 2 of 55
Participant 3 of 55
Participant 4 of 55
Participant 5 of 55
Participant 6 of 55
Participant 7 of 55
Participant 8 of 55
Participant 9 of 55
Participant 10 of 55
Participant 11 of 55
Participant 12 of 55
Participant 13 of 55
Participant 14 of 55
Participant 15 of 55
Participant 16 of 55
Participant 17 of 55
Participant 18 of 55
Participant 19 of 55
Participant 20 of 55
Participant 21 of 55
Participant 22 of 55
Participant 23 of 55
Participant 24 of 55
Participant 25 of 55
Participant 26 of 55
Participant 27 of 55
Participant 28 of 55
Participant 29 of 55
Participant 30 of 55
Participant 31 of 55
Participant 32 of 55
Participant 33 of 55
Participant 34 of 55
Participant 35 of 55
Participant 36 of 55
Participant 37 of 55
Participant 38 of 55
Participant 39 of 55
Participant 40 of 55
Participant 41 of 55
Participant 42 of 55
Participant 43 of 55
Participant 44 of 55
Participant 45 of 55
Participant 46 of 55
Participant 47 of 55
Participant 48 of 55
P

In [56]:
results_saving_dir = folder_name + '/test_best_models'
results_final_test = pd.read_csv(f'{results_saving_dir}/results_final.csv')
columns =  ['convergence_point', 'initial_sleep_onset_threshold', 'sliding_window_size',  'one_crossing_percentage_smoothed',
       'mean_time_of_prediction_smoothed',
       'std_time_of_prediction_smoothed',  'f1_weighted', 'f1_pre_sleep',
       'f1_awake', 'precision',
       'recall', 'auc']
results_final_test_selected = results_final_test[columns]

sorted_results = results_final_test_selected.sort_values(by=['one_crossing_percentage_smoothed'], ascending=False)
sorted_results = sorted_results.round(2)
headers = ['t_{PS}', 't_0_{PS}', 't_{SW}', 'R_ps', 'Mean(t_{pred})', 'STD(t_{pred})', 'F1_W', 'F1_{PS}','F1_{AW}', 'P', 'S', 'AUC']
sorted_results.columns = headers

sorted_results.to_latex(f'{folder_name}/results_test_best.tex', index=False, float_format="%.2f")
print(sorted_results)

<ipython-input-56-a4d7857caf55>:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  sorted_results.to_latex(f'{folder_name}/results_test_best.tex', index=False, float_format="%.2f")


   t_{PS}  t_0_{PS}  t_{SW}   R_ps  Mean(t_{pred})  STD(t_{pred})  F1_W  \
0       4         2     1.5  34.55            9.12           8.83  0.80   
1      12         6     1.0  20.00           23.29           4.59  0.31   

   F1_{PS}  F1_{AW}     P     S   AUC  
0     0.85     0.28  0.19  0.53  0.74  
1     0.16     0.56  0.40  0.97  0.64  
